<h1>Analysis overview</h1>

In [1]:
import warnings
    
# Import general modules
import os
import re
from itertools import cycle
from multiprocessing import Pool

# Import HTML/js libraries
from IPython.display import display, Javascript, HTML, Math, Latex
import jinja2

# Import data science modules
import numpy as np
import scipy
import scipy.stats
from scipy import interp, ndimage
import pandas as pd
import nibabel as nib
import seaborn as sns

# Import graphing modules
%matplotlib inline  
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')

# Import machine learning modules
from sklearn.ensemble import *
from sklearn.metrics import *
from sklearn.linear_model import *
from sklearn.model_selection import *
from sklearn.feature_selection import *
from sklearn.preprocessing import *
from sklearn.tree import *
from sklearn.externals import joblib
from sklearn.svm import *
from sklearn.decomposition import *
import sklearn.utils
from sklearn.utils import *

from collections import defaultdict
from collections import OrderedDict

# Import R - py module
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri, numpy2ri
from rpy2.robjects.packages import importr
pandas2ri.activate()
numpy2ri.activate()

def _helper(job):
    job_iter,train,test,X1,y1,XN,yN,classifier1,classifierN = job
    if(job_iter %1000 == 0):
        print job_iter
    X1_train = X1[train]
    y1_train = y1[train]
    X1_test = X1[test]
    y1_test = y1[test]

    XN_train = XN[train]
    yN_train = yN[train]
    XN_test = XN[test]
    yN_test = yN[test]

    probas_1 = classifier1.fit(X1_train, y1_train).predict_proba(X1_test)
    probas_N = classifierN.fit(XN_train, yN_train).predict_proba(XN_test)

    return [probas_1,y1_test,probas_N,yN_test,classifier1.oob_score_,classifierN.oob_score_]

ssp_labels = [u'*',u'Angular Gyrus',u'Anterior Cingulate',u'Caudate', u'Cerebellar Lingual',u'Cerebellar Tonsil',u'Cingulate Gyrus',u'Culmen', u'Culmen of Vermis',u'Cuneus',u'Declive',u'Declive of Vermis', u'Extra-Nuclear',u'Fusiform Gyrus',u'Inferior Frontal Gyrus', u'Inferior Occipital Gyrus',u'Inferior Parietal Lobule', u'Inferior Semi-Lunar Lobule',u'Inferior Temporal Gyrus',u'Lingual Gyrus', u'Medial Frontal Gyrus',u'Middle Frontal Gyrus',u'Middle Occipital Gyrus', u'Middle Temporal Gyrus',u'Nodule',u'Orbital Gyrus',u'Paracentral Lobule', u'Parahippocampal Gyrus',u'Postcentral Gyrus',u'Posterior Cingulate', u'Precentral Gyrus',u'Precuneus',u'Pyramis',u'Pyramis of Vermis', u'Rectal Gyrus',u'Subcallosal Gyrus',u'Superior Frontal Gyrus', u'Superior Occipital Gyrus',u'Superior Parietal Lobule', u'Superior Temporal Gyrus',u'Supramarginal Gyrus',u'Thalamus', u'Transverse Temporal Gyrus',u'Tuber',u'Tuber of Vermis',u'Uncus',u'Uvula', u'Uvula of Vermis']
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [2]:
RANDOM_STATE = 929

# Model parameters
NUM_BOOTSTRAPS = 5000
max_depth = 2

n_proc = 32
pool = Pool(n_proc)

<h1>Demographics (Table 1)</h1>

In [3]:
from printTable1 import *
results = compute_table1()
results

,Demographic Feature,Complete Seizure Freedom (Engel IA),Good Surgical Outcome (Engel IB-D),"Poor Surgical Outcome (Engel II, III, IV)",p-value (Developmental),Complete Seizure Freedom (Engel IA),Good Surgical Outcome (Engel IB-D),"Poor Surgical Outcome (Engel II, III, IV)",p-value (Validation)
0,Total number of subjects,25,27,25,NaN,7,6,6,NaN
1,Age at surgery,-,-,-,NaN,-,-,-,NaN
2,-,38.45 p/m 12.01,39.19 p/m 12.14,35.35 p/m 11.09,,37.10 p/m 16.03,29.01 p/m 13.49,52.93 p/m 6.92,
3,Gender,-,-,-,0.628779,-,-,-,0.303835
4,Male,9,10,12,-,2,0,2,-
5,Female,16,17,13,-,5,6,4,-
6,Resected Regions,-,-,-,0.539441,-,-,-,0.324409
7,LTL,14,11,14,-,2,4,2,-
8,RTL,11,14,8,-,5,2,2,-
9,LFL/RFL,0,1,2,-,0,0,1,-


<h1>Prediction of seizure recurrence: Engel IA vs. IB-D, II-IV</h1>

<h2> Feature selection </h2>

In [ ]:
#Feature Selection for Model B

# Parameters for feature selection
n_repeats = 100
num_cv = 5

# Load data
df = pd.read_csv('../data/qPET_feature_matrix_clinical_3dssp.csv')
df_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_3dssp.csv')

# Generate feature matrix and target vectors
X = np.array(df[df.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X_labels = np.array(df.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))

# Generate output labels
y = np.array(df.outcomeLatest)
y[y<2] = 0
y[y>=2] = 1


all_features = []
for iter_id in range(n_repeats):
    X_resample, y_resample = sklearn.utils.resample(X,y,n_samples=int(0.7*X.shape[0]),replace=False, random_state=RANDOM_STATE)
    fdr = SelectKBest(f_classif,k=int(n_repeats/2))
    fdr.fit(X_resample, y_resample)
    for index in fdr.get_support(indices=True):
        all_features.append(index)
all_features =  np.sort(all_features)

ft_counts = {}
for ft in all_features:
    try:
        ft_counts[ft] += 1
    except KeyError:
        ft_counts[ft] = 1
SSP_best_features1 = []
for ft,ft_count in ft_counts.items():
    if ft_count > n_repeats/2:
        SSP_best_features1.append(ft)

estimator = RandomForestClassifier(n_estimators=100, max_depth=max_depth, random_state=RANDOM_STATE)
selector = RFECV(estimator, step=1, cv=num_cv)
selector = selector.fit(X,y)
SSP_best_features2 = np.where(selector.support_)[0]

SSP_features = list(set(SSP_best_features1) & set(SSP_best_features2))
print sorted(SSP_features)


In [ ]:
#Feature Selection for Model C

# Load data
df = pd.read_csv('../data/qPET_feature_matrix_clinical_voxel_ai.csv')
df_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_voxel_ai.csv')

# Generate feature matrix and target vectors
X = np.array(df[df.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X_labels = np.array(df.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
# Generate output labels
y = np.array(df.outcomeLatest)
y[y<2] = 0
y[y>=2] = 1

all_features = []
for iter_id in range(n_repeats):
    X_resample, y_resample = sklearn.utils.resample(X,y,n_samples=int(0.7*X.shape[0]),replace=False, random_state=RANDOM_STATE)
    fdr = SelectKBest(f_classif,k=int(n_repeats/2))
    fdr.fit(X_resample, y_resample)
    for index in fdr.get_support(indices=True):
        all_features.append(index)
all_features =  np.sort(all_features)

ft_counts = {}
for ft in all_features:
    try:
        ft_counts[ft] += 1
    except KeyError:
        ft_counts[ft] = 1
ASYMM_best_features1 = []
for ft,ft_count in ft_counts.items():
    if ft_count > n_repeats/2:
        ASYMM_best_features1.append(ft)

estimator = RandomForestClassifier(n_estimators=100, max_depth=max_depth, random_state=RANDOM_STATE)
selector = RFECV(estimator, step=1, cv=num_cv)
selector = selector.fit(X,y)
ASYMM_best_features2 = np.where(selector.support_)[0]

ASYMM_features = list(set(ASYMM_best_features1) & set(ASYMM_best_features2))
print sorted(ASYMM_features) 

In [4]:
SSP_features = [1, 5, 32, 33, 41, 44, 50, 51, 59, 83, 108, 109, 112, 117, 126, 127, 135, 136]
ASYMM_features = [27, 42, 44, 49, 65, 73, 144, 217, 234, 236, 339, 341, 352]

In [ ]:
len(SSP_features), len(ASYMM_features)

<h2>Determine number of estimators</h2>

In [ ]:
# Load data for CLINICAL ONLY
dfA = pd.read_csv('../data/qPET_feature_matrix_clinical_only.csv')
dfA_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_only.csv')

# Generate feature matrix for training and testing data
# Training
X1 = np.array(dfA[dfA.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X1_labels = np.array(dfA.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))

# Load data for SSP ONLY
dfB = pd.read_csv('../data/qPET_feature_matrix_clinical_3dssp.csv')
dfB_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_3dssp.csv')

# Generate feature matrix for training and testing data
# Training
X2 = np.array(dfB[dfB.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X2_labels = np.array(dfB.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X2 = X2[:,SSP_features]
X2_labels = X2_labels[SSP_features]

# Load data for ASYMMETRY ONLY
dfC = pd.read_csv('../data/qPET_feature_matrix_clinical_voxel_ai.csv')
dfC_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_voxel_ai.csv')

# Generate feature matrix for training testing data
# Training
X3 = np.array(dfC[dfC.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X3_labels = np.array(dfC.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X3 = X3[:,ASYMM_features]
X3_labels = X3_labels[ASYMM_features]

# Generate outcome variable
outcome_threshold = 2 # i.e. Engel 1B
y = np.array(dfA.outcomeLatest)
y_test = np.array(dfA_test.outcomeLatest)
y[y<outcome_threshold] = 0
y[y>=outcome_threshold] = 1
y_test[y_test<outcome_threshold] = 0
y_test[y_test>=outcome_threshold] = 1


XA = X1
XA_labels = X1_labels
yA = y

XB = np.hstack((X1,X2))
XB_labels = np.hstack((X1_labels,X2_labels))
yB = y

yC = y
XC = np.hstack((X1,X3))
XC_labels = np.hstack((X1_labels,X3_labels))


min_estimators = 15
max_estimators = 1000

clfA = RandomForestClassifier(warm_start=True, max_features=None, 
                             oob_score=True, max_depth=max_depth, 
                             random_state=RANDOM_STATE)
error_rate = []
for i in range(min_estimators, max_estimators+1):
    clfA.set_params(n_estimators=i)
    clfA.fit(XA,yA)
    oob_error = 1-clfA.oob_score_
    error_rate.append((i, oob_error))
error_rate = np.array(error_rate)
plt.plot(error_rate[:,0], error_rate[:,1])
plt.xlim(min_estimators, max_estimators)
plt.title('OOB Error versus Number of Estimators for Clinical Only')
plt.xlabel('n_estimators')
plt.ylabel('OOB error rate')
plt.show()
nest = error_rate[:,0] 
err = error_rate[:,1]
n_estimatorsA = int(nest[np.where(err == np.min(err))[0][0]])

clfB = RandomForestClassifier(warm_start=True, max_features=None, 
                             oob_score=True, max_depth=max_depth, 
                             random_state=RANDOM_STATE)
error_rate = []
for i in range(min_estimators, max_estimators+1):
    clfB.set_params(n_estimators=i)
    clfB.fit(XB,yB)
    oob_error = 1-clfB.oob_score_
    error_rate.append((i, oob_error))
error_rate = np.array(error_rate)
plt.plot(error_rate[:,0], error_rate[:,1])
plt.xlim(min_estimators, max_estimators)
plt.title('OOB Error versus Number of Estimators for Clinical Only')
plt.xlabel('n_estimators')
plt.ylabel('OOB error rate')
plt.show()
nest = error_rate[:,0] 
err = error_rate[:,1]
n_estimatorsB = int(nest[np.where(err == np.min(err))[0][0]])

clfC = RandomForestClassifier(warm_start=True, max_features=None, 
                             oob_score=True, max_depth=max_depth, 
                             random_state=RANDOM_STATE)
error_rate = []
for i in range(min_estimators, max_estimators+1):
    clfC.set_params(n_estimators=i)
    clfC.fit(XC,yC)
    oob_error = 1-clfC.oob_score_
    error_rate.append((i, oob_error))
error_rate = np.array(error_rate)
plt.plot(error_rate[:,0], error_rate[:,1])
plt.xlim(min_estimators, max_estimators)
plt.title('OOB Error versus Number of Estimators for Clinical Only')
plt.xlabel('n_estimators')
plt.ylabel('OOB error rate')
plt.show()
nest = error_rate[:,0] 
err = error_rate[:,1]
n_estimatorsC = int(nest[np.where(err == np.min(err))[0][0]])

print n_estimatorsA, n_estimatorsB, n_estimatorsC

<h2>Measure cross-validation scores</h2>

This section ...

In [ ]:
# Preprocessing
classifierA = RandomForestClassifier(n_estimators=n_estimatorsA, max_depth=max_depth, random_state=RANDOM_STATE,oob_score=True)
classifierB = RandomForestClassifier(n_estimators=n_estimatorsB, max_depth=max_depth, random_state=RANDOM_STATE, oob_score=True)
classifierC = RandomForestClassifier(n_estimators=n_estimatorsC, max_depth=max_depth, random_state=RANDOM_STATE, oob_score=True)

sss = StratifiedShuffleSplit(n_splits=NUM_BOOTSTRAPS, test_size=0.2, random_state=RANDOM_STATE)

print 'Generating feature matrices ...'
# Load data for CLINICAL ONLY
dfA = pd.read_csv('../data/qPET_feature_matrix_clinical_only.csv')
dfA_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_only.csv')

# Generate feature matrix for training and testing data
# Training
X1 = np.array(dfA[dfA.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X1_labels = np.array(dfA.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
# Testing
X1_test = np.array(dfA_test[dfA_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X1_test_labels = np.array(dfA_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))

# Load data for SSP ONLY
dfB = pd.read_csv('../data/qPET_feature_matrix_clinical_3dssp.csv')
dfB_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_3dssp.csv')

# Generate feature matrix for training and testing data
# Training
X2 = np.array(dfB[dfB.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X2_labels = np.array(dfB.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X2 = X2[:,SSP_features]
X2_labels = X2_labels[SSP_features]
# Testing
X2_test = np.array(dfB_test[dfB_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X2_test_labels = np.array(dfB_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X2_test = X2_test[:,SSP_features]
X2_test_labels = X2_test_labels[SSP_features]

# Load data for ASYMMETRY ONLY
dfC = pd.read_csv('../data/qPET_feature_matrix_clinical_voxel_ai.csv')
dfC_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_voxel_ai.csv')

# Generate feature matrix for training testing data
# Training
X3 = np.array(dfC[dfC.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X3_labels = np.array(dfC.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X3 = X3[:,ASYMM_features]
X3_labels = X3_labels[ASYMM_features]
# Testing
X3_test = np.array(dfC_test[dfC_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X3_test_labels = np.array(dfC_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X3_test = X3_test[:,ASYMM_features]
X3_test_labels = X3_test_labels[ASYMM_features]

# Generate outcome variable
outcome_threshold = 2 # i.e. Engel 1B
y = np.array(dfA.outcomeLatest)
y_test = np.array(dfA_test.outcomeLatest)
y[y<outcome_threshold] = 0
y[y>=outcome_threshold] = 1
y_test[y_test<outcome_threshold] = 0
y_test[y_test>=outcome_threshold] = 1

In [ ]:
columns=['Clinical Variable(s)','CV 5-Fold AUC_1','Out Of Bag (OOB1) Error',
         'Quantitative Variables','CV 5-Fold AUC_2','Out Of Bag (OOB2) Error',
         'AUC Difference 95% C.I.','OOB Difference 95% C.I.'
        ]
results = pd.DataFrame(
    {'Clinical Variable(s)':[],
     'CV 5-Fold AUC_1':[],
     'Out Of Bag (OOB1) Error':[],
     'Quantitative Variables':[],
     'CV 5-Fold AUC_2':[],
     'Out Of Bag (OOB2) Error':[],
     'AUC Difference 95% C.I.':[],
     'OOB Difference 95% C.I.':[]
             },
    columns=columns
)

OPTIONS = {
    1:[np.arange(20),'EEG+MRI+PET'],
    2:[np.arange(12),'EEG'],
    3:[np.arange(12,14),'MRI'],
    4:[np.arange(13,20),'PET'],
}

for OPTION,(clinical_variable_idx,variables_list) in OPTIONS.items():
    # Generate feature matrix for CLINICAL ONLY
    XA = X1[:,clinical_variable_idx]
    XA_labels = X1_labels[clinical_variable_idx]
    yA = y

    # Generate feature matrix for SSP alone
    XB = X2
    XB_labels = X2_labels
    XB_test = X2_test
    XB_test_labels = X2_test_labels
    yB = y

    # Generate bootstrap jobs
    jobs = []
    out = []
    for job_iter,(train, test) in enumerate(sss.split(XA,yA)):
        jobs.append((job_iter,train,test,XA,yA,XB,yB,classifierA,classifierB))
    # Run all jobs
    return_list = pool.map(_helper,jobs)
    # Compute OOB/AUC for bootstraps
    for res in return_list:
        probas_A,yA_test,probas_B,yB_test,classifierA_oob_score_,classifierB_oob_score_ = res
        fprA, tprA, thresholds = roc_curve(yA_test, probas_A[:, 1])
        fprB, tprB, thresholds = roc_curve(yB_test, probas_B[:, 1])
        roc_aucA = auc(fprA, tprA)
        roc_aucB = auc(fprB, tprB)
        out.append([roc_aucA,roc_aucB,classifierA_oob_score_,classifierB_oob_score_])
    out21 = np.array(out)

    # Generate feature matrix for SSP and CLINICAL
    XB = np.hstack((X1,X2))
    XB_labels = np.hstack((X1_labels,X2_labels))
    XB_test = np.hstack((X1_test,X2_test))
    XB_test_labels = np.hstack((X1_test_labels,X2_test_labels))

    # Generate bootstrap jobs
    jobs = []
    out = []
    for job_iter,(train, test) in enumerate(sss.split(XA,yA)):
        jobs.append((job_iter,train,test,XA,yA,XB,yB,classifierA,classifierB))
    # Run all jobs
    return_list = pool.map(_helper,jobs)
    # Compute OOB/AUC for bootstraps
    for res in return_list:
        probas_A,yA_test,probas_B,yB_test,classifierA_oob_score_,classifierB_oob_score_ = res
        fprA, tprA, thresholds = roc_curve(yA_test, probas_A[:, 1])
        fprB, tprB, thresholds = roc_curve(yB_test, probas_B[:, 1])
        roc_aucA = auc(fprA, tprA)
        roc_aucB = auc(fprB, tprB)
        out.append([roc_aucA,roc_aucB,classifierA_oob_score_,classifierB_oob_score_])
    out31 = np.array(out)

    # Compute statistics
    AUC1 = np.mean(out21[:,0])
    OOB1 = np.mean(out21[:,2])
    AUC2 = np.mean(out21[:,1])
    OOB2 = np.mean(out21[:,3])
    AUC3 = np.mean(out31[:,1])
    OOB3 = np.mean(out31[:,3])
    CI_AUC2_minus_AUC1 = tuple(map(lambda x: np.percentile(out21[:,1] - out21[:,0],x), [2.5, 97.5]))
    CI_AUC3_minus_AUC1 = tuple(map(lambda x: np.percentile(out31[:,1] - out31[:,0],x), [2.5, 97.5]))
    CI_OOB2_minus_OOB1 = tuple(map(lambda x: np.percentile(out21[:,3] - out21[:,2],x), [2.5, 97.5]))
    CI_OOB3_minus_OOB1 = tuple(map(lambda x: np.percentile(out31[:,3] - out31[:,2],x), [2.5, 97.5]))


    results = results.append(
        pd.DataFrame(
            [
                [variables_list,AUC1,OOB1,'%s+SSP'%(variables_list), AUC2, OOB2, CI_AUC2_minus_AUC1, CI_OOB2_minus_OOB1],
                [variables_list,AUC1,OOB1,'SSP only', AUC3, OOB3, CI_AUC3_minus_AUC1, CI_OOB3_minus_OOB1]
            ],
            columns=columns
        ),
        ignore_index=True
    )
cm = sns.light_palette("green",as_cmap=True)
results.style.background_gradient(cmap=cm,high=1.0)

In [ ]:
columns=['Clinical Variable(s)','CV 5-Fold AUC_1','Out Of Bag (OOB1) Error',
         'Quantitative Variables','CV 5-Fold AUC_2','Out Of Bag (OOB2) Error',
         'AUC Difference 95% C.I.','OOB Difference 95% C.I.'
        ]
results = pd.DataFrame(
    {'Clinical Variable(s)':[],
     'CV 5-Fold AUC_1':[],
     'Out Of Bag (OOB1) Error':[],
     'Quantitative Variables':[],
     'CV 5-Fold AUC_2':[],
     'Out Of Bag (OOB2) Error':[],
     'AUC Difference 95% C.I.':[],
     'OOB Difference 95% C.I.':[]
             },
    columns=columns
)

OPTIONS = {
    1:[np.arange(20),'EEG+MRI+PET'],
    2:[np.arange(12),'EEG'],
    3:[np.arange(12,14),'MRI'],
    4:[np.arange(13,20),'PET'],
}

for OPTION,(clinical_variable_idx,variables_list) in OPTIONS.items():
    # Generate feature matrix for CLINICAL ONLY
    XA = X1[:,clinical_variable_idx]
    XA_labels = X1_labels[clinical_variable_idx]
    yA = y

    # Generate feature matrix for SSP alone
    XC = X3
    XC_labels = X3_labels
    XC_test = X3_test
    XC_test_labels = X3_test_labels
    yC = y

    # Generate bootstrap jobs
    jobs = []
    out = []
    for job_iter,(train, test) in enumerate(sss.split(XA,yA)):
        jobs.append((job_iter,train,test,XA,yA,XC,yC,classifierA,classifierC))
    # Run all jobs
    return_list = pool.map(_helper,jobs)
    # Compute OOB/AUC for bootstraps
    for res in return_list:
        probas_A,yA_test,probas_C,yC_test,classifierA_oob_score_,classifierC_oob_score_ = res
        fprA, tprA, thresholds = roc_curve(yA_test, probas_A[:, 1])
        fprC, tprC, thresholds = roc_curve(yC_test, probas_C[:, 1])
        roc_aucA = auc(fprA, tprA)
        roc_aucC = auc(fprC, tprC)
        out.append([roc_aucA,roc_aucC,classifierA_oob_score_,classifierC_oob_score_])
    out21 = np.array(out)

    # Generate feature matrix for SSP and CLINICAL
    XC = np.hstack((X1,X3))
    XC_labels = np.hstack((X1_labels,X3_labels))
    XC_test = np.hstack((X1_test,X3_test))
    XC_test_labels = np.hstack((X1_test_labels,X3_test_labels))

    # Generate bootstrap jobs
    jobs = []
    out = []
    for job_iter,(train, test) in enumerate(sss.split(XA,yA)):
        jobs.append((job_iter,train,test,XA,yA,XC,yB,classifierA,classifierB))
    # Run all jobs
    return_list = pool.map(_helper,jobs)
    # Compute OOB/AUC for bootstraps
    for res in return_list:
        probas_A,yA_test,probas_C,yC_test,classifierA_oob_score_,classifierC_oob_score_ = res
        fprA, tprA, thresholds = roc_curve(yA_test, probas_A[:, 1])
        fprC, tprC, thresholds = roc_curve(yC_test, probas_C[:, 1])
        roc_aucA = auc(fprA, tprA)
        roc_aucC = auc(fprC, tprC)
        out.append([roc_aucA,roc_aucC,classifierA_oob_score_,classifierC_oob_score_])
    out31 = np.array(out)

    # Compute statistics
    AUC1 = np.mean(out21[:,0])
    OOB1 = np.mean(out21[:,2])
    AUC2 = np.mean(out21[:,1])
    OOB2 = np.mean(out21[:,3])
    AUC3 = np.mean(out31[:,1])
    OOB3 = np.mean(out31[:,3])
    CI_AUC2_minus_AUC1 = tuple(map(lambda x: np.percentile(out21[:,1] - out21[:,0],x), [2.5, 97.5]))
    CI_AUC3_minus_AUC1 = tuple(map(lambda x: np.percentile(out31[:,1] - out31[:,0],x), [2.5, 97.5]))
    CI_OOB2_minus_OOB1 = tuple(map(lambda x: np.percentile(out21[:,3] - out21[:,2],x), [2.5, 97.5]))
    CI_OOB3_minus_OOB1 = tuple(map(lambda x: np.percentile(out31[:,3] - out31[:,2],x), [2.5, 97.5]))


    results = results.append(
        pd.DataFrame(
            [
                [variables_list,AUC1,OOB1,'%s+Asymmetry'%(variables_list), AUC2, OOB2, CI_AUC2_minus_AUC1, CI_OOB2_minus_OOB1],
                [variables_list,AUC1,OOB1,'Asymmetry only', AUC3, OOB3, CI_AUC3_minus_AUC1, CI_OOB3_minus_OOB1]
            ],
            columns=columns
        ),
        ignore_index=True
    )
cm = sns.light_palette("green",as_cmap=True)
results.style.background_gradient(cmap=cm,high=1.0)

<h2>Feature Importances (Table 3)</h2>

In [5]:
# Create feature matrix for training and testing
# Load data for CLINICAL ONLY
dfA = pd.read_csv('../data/qPET_feature_matrix_clinical_only.csv')
dfA_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_only.csv')

# Generate feature matrix for training and testing data
# Training
X1 = np.array(dfA[dfA.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X1_labels = np.array(dfA.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
# Testing
X1_test = np.array(dfA_test[dfA_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X1_test_labels = np.array(dfA_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))

# Load data for SSP ONLY
dfB = pd.read_csv('../data/qPET_feature_matrix_clinical_3dssp.csv')
dfB_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_3dssp.csv')

# Generate feature matrix for training and testing data
# Training
X2 = np.array(dfB[dfB.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X2_labels = np.array(dfB.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X2 = X2[:,SSP_features]
X2_labels = X2_labels[SSP_features]
# Testing
X2_test = np.array(dfB_test[dfB_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X2_test_labels = np.array(dfB_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X2_test = X2_test[:,SSP_features]
X2_test_labels = X2_test_labels[SSP_features]

# Load data for ASYMMETRY ONLY
dfC = pd.read_csv('../data/qPET_feature_matrix_clinical_voxel_ai.csv')
dfC_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_voxel_ai.csv')

# Generate feature matrix for training testing data
# Training
X3 = np.array(dfC[dfC.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X3_labels = np.array(dfC.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X3 = X3[:,ASYMM_features]
X3_labels = X3_labels[ASYMM_features]
# Testing
X3_test = np.array(dfC_test[dfC_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X3_test_labels = np.array(dfC_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X3_test = X3_test[:,ASYMM_features]
X3_test_labels = X3_test_labels[ASYMM_features]

# Generate outcome variable
# outcome_threshold = 2 # i.e. Engel 1B
y = np.array(dfA.outcomeLatest)
y_test = np.array(dfA_test.outcomeLatest)
# y[y<outcome_threshold] = 0
# y[y>=outcome_threshold] = 1
# y_test[y_test<outcome_threshold] = 0
# y_test[y_test>=outcome_threshold] = 1

# Run predictions
# Generate feature matrix for CLINICAL ONLY
XA = X1
XA_labels = X1_labels
XA_test = X1_test
XA_test_labels = X1_test_labels
yA = y

# Generate feature matrix for SSP and CLINICAL
XB = np.hstack((X1,X2))
XB_labels = np.hstack((X1_labels,X2_labels))
XB_test = np.hstack((X1_test,X2_test))
XB_test_labels = np.hstack((X1_test_labels,X2_test_labels))
yB = y

# Generate feature matrix for SSP and CLINICAL
XC = np.hstack((X1,X3))
XC_labels = np.hstack((X1_labels,X3_labels))
XC_test = np.hstack((X1_test,X3_test))
XC_test_labels = np.hstack((X1_test_labels,X3_test_labels))
yC = y


In [6]:
robjects.globalenv['RANDOM_STATE'] = RANDOM_STATE

# Generate final form of Table 2
columns=[
    'Feature',
    'Feature Importance',
    'Univariate Odds Ratio per Unit\n Increase in Seizure Recurrence [95% CI, p]'
        ]
results = pd.DataFrame(
    {'Feature':[],
     'Feature Importance':[],
     'Univariate Odds Ratio per Unit\n Increase in Seizure Recurrence [95% CI, p]':[]
             },
    columns=columns
)

for X,y, X_labels in [(XA,yA,XA_labels), (XB,yB,XB_labels), (XC,yC,XC_labels)]:
    # Run feature importance (IncMSE) for random forests
    robjects.globalenv['X'] = X
    robjects.globalenv['y'] = y
    summary = robjects.r['summary']

    robjects.r('''
        set.seed(RANDOM_STATE)
        library(randomForest)        
        rf <- randomForest(y ~ ., data=X, importance=TRUE)
        X_importances <- importance(rf)    
    ''')
    X_importances = robjects.r['X_importances']
    feature_order = np.argsort(-X_importances[:,0])

    # Run univariate odds ratio computation
    for feature in feature_order:
        robjects.globalenv['feature'] = X[:,feature]    
        robjects.r('''        
            set.seed(RANDOM_STATE)
            require(MASS)        
            myfit <- polr(as.factor(y) ~ feature, Hess=TRUE)
            (ctable <- coef(summary(myfit)))            
            p <- pnorm(abs(ctable[,"t value"]), lower.tail=FALSE)*2
            (ctable <- cbind(ctable, "p value" = p))
            (ci <- confint(myfit))
            myfit_ci <-exp(cbind(OR = coef(myfit), ci))            
        ''')
        myfit = robjects.r['myfit']
        myfit_ci = robjects.r['myfit_ci']
        pval = robjects.r['p'][0]
        label = X_labels[feature]
        try:
            label = label.replace(label.split('_')[-1],ssp_labels[int(label.split('_')[-1])])
        except Exception:
            pass

        if X_importances[feature,0] <= 0.0:
            continue
        
        if pval < 0.001:
            pval = '%0.3f ***'%pval
        elif pval < 0.01:
            pval = '%0.3f **'%pval
        elif pval < 0.05:
            pval = '%0.3f *'%pval
        elif pval < 0.1:
            pval = '%0.3f .'%pval
        else:
            pval = '%0.2f'%pval
        
        results = results.append(
            pd.DataFrame(
                [
                    [
                        label,
                        '%0.2f'%X_importances[feature,0], 
                        '%0.2f [%0.2f - %0.2f, %s]'%(
                            myfit_ci[0,0],
                            myfit_ci[0,1], myfit_ci[1,1], 
                            pval
                        )
                    ]
                ],
                columns=columns
            ),
            ignore_index=True
        )
    results = results.append(
        pd.DataFrame(
            [
                [
                    '',
                    '', 
                    ''
                ]
            ],
            columns=columns
        ),
        ignore_index=True
    )
# cm = sns.light_palette("green",as_cmap=True)
# results.style.background_gradient(cmap=cm,high=1.0,low=0.0)
display(results)
results.to_csv('results_Section3.csv')

/gdrive/public/USERS/lkini/thesis/ec2/anaconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: randomForest 4.6-12

  warnings.warn(x, RRuntimeWarning)
/gdrive/public/USERS/lkini/thesis/ec2/anaconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Type rfNews() to see new features/changes/bug fixes.

  warnings.warn(x, RRuntimeWarning)
/gdrive/public/USERS/lkini/thesis/ec2/anaconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Loading required package: MASS

  warnings.warn(x, RRuntimeWarning)
/gdrive/public/USERS/lkini/thesis/ec2/anaconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Waiting for profiling to be done...

  warnings.warn(x, RRuntimeWarning)


,Feature,Feature Importance,"Univariate Odds Ratio per Unit Increase in Seizure Recurrence [95% CI, p]"
0,petclass_S,10.04,"0.29 [0.10 - 0.77, 0.016 *]"
1,eeg_D,5.28,"3.56 [0.76 - 17.79, 0.11]"
2,eeg_LTL+,4.89,"3.00 [0.72 - 12.90, 0.13]"
3,eeg_RTL+,4.82,"0.40 [0.05 - 2.59, 0.33]"
4,petclass_M,3.61,"3.36 [1.16 - 10.00, 0.026 *]"
5,petclass_R,2.02,"0.71 [0.32 - 1.60, 0.41]"
6,lesional_NL,1.08,"2.35 [1.04 - 5.39, 0.041 *]"
7,,,
8,eeg_LTL+,4.02,"3.00 [0.72 - 12.90, 0.13]"
9,ipsi_z_ai_3dssp_Angular Gyrus,3.96,"0.53 [0.33 - 0.82, 0.006 **]"


<h2>Validation on testing set (Table 4)</h2>

In [ ]:
# Create feature matrix for training and testing
# Load data for CLINICAL ONLY
dfA = pd.read_csv('../data/qPET_feature_matrix_clinical_only.csv')
dfA_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_only.csv')

# Generate feature matrix for training and testing data
# Training
X1 = np.array(dfA[dfA.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X1_labels = np.array(dfA.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
# Testing
X1_test = np.array(dfA_test[dfA_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X1_test_labels = np.array(dfA_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))

# Load data for SSP ONLY
dfB = pd.read_csv('../data/qPET_feature_matrix_clinical_3dssp.csv')
dfB_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_3dssp.csv')

# Generate feature matrix for training and testing data
# Training
X2 = np.array(dfB[dfB.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X2_labels = np.array(dfB.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X2 = X2[:,SSP_features]
X2_labels = X2_labels[SSP_features]
# Testing
X2_test = np.array(dfB_test[dfB_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X2_test_labels = np.array(dfB_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X2_test = X2_test[:,SSP_features]
X2_test_labels = X2_test_labels[SSP_features]

# Load data for ASYMMETRY ONLY
dfC = pd.read_csv('../data/qPET_feature_matrix_clinical_voxel_ai.csv')
dfC_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_voxel_ai.csv')

# Generate feature matrix for training testing data
# Training
X3 = np.array(dfC[dfC.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X3_labels = np.array(dfC.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X3 = X3[:,ASYMM_features]
X3_labels = X3_labels[ASYMM_features]
# Testing
X3_test = np.array(dfC_test[dfC_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X3_test_labels = np.array(dfC_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X3_test = X3_test[:,ASYMM_features]
X3_test_labels = X3_test_labels[ASYMM_features]

# Generate outcome variable
outcome_threshold = 2 # i.e. Engel 1B
y = np.array(dfA.outcomeLatest)
y_test = np.array(dfA_test.outcomeLatest)
y[y<outcome_threshold] = 0
y[y>=outcome_threshold] = 1
y_test[y_test<outcome_threshold] = 0
y_test[y_test>=outcome_threshold] = 1

# Run predictions
# Generate feature matrix for CLINICAL ONLY
XA = X1
XA_labels = X1_labels
XA_test = X1_test
XA_test_labels = X1_test_labels

# Generate feature matrix for SSP and CLINICAL
XB = np.hstack((X1,X2))
XB_labels = np.hstack((X1_labels,X2_labels))
XB_test = np.hstack((X1_test,X2_test))
XB_test_labels = np.hstack((X1_test_labels,X2_test_labels))

# Generate feature matrix for SSP and CLINICAL
XC = np.hstack((X1,X3))
XC_labels = np.hstack((X1_labels,X3_labels))
XC_test = np.hstack((X1_test,X3_test))
XC_test_labels = np.hstack((X1_test_labels,X3_test_labels))

# Load library in R
pROC = importr('pROC')

# Train classifier and apply to validation test set
classifierA = RandomForestClassifier(
    n_estimators=n_estimatorsA, 
    max_depth=max_depth, 
    random_state=RANDOM_STATE,
    oob_score=True)
classifierB = RandomForestClassifier(
    n_estimators=n_estimatorsB, 
    max_depth=max_depth, 
    random_state=RANDOM_STATE,
    oob_score=True)
classifierC = RandomForestClassifier(
    n_estimators=n_estimatorsC, 
    max_depth=max_depth, 
    random_state=RANDOM_STATE,
    oob_score=True)

probas = classifierA.fit(XA, y).predict_proba(XA_test)
predsA = robjects.FloatVector(probas[:,1])
labelsA = robjects.IntVector(y_test)
probas = classifierB.fit(XB, y).predict_proba(XB_test)
predsB = robjects.FloatVector(probas[:,1])
labelsB = robjects.IntVector(y_test)
probas = classifierC.fit(XC, y).predict_proba(XC_test)
predsC = robjects.FloatVector(probas[:,1])
labelsC = robjects.IntVector(y_test)


# Copy from python workspace to R workspace
robjects.globalenv["predsA"] = predsA
robjects.globalenv["labelsA"] = labelsA
robjects.globalenv["predsB"] = predsB
robjects.globalenv["labelsB"] = labelsB
robjects.globalenv["predsC"] = predsC
robjects.globalenv["labelsC"] = labelsC

# Run pROC.roc and pROC.roc.test in R
robjects.r('''
    predsA<-as.numeric(unlist(predsA))
    labelsA<-as.numeric(unlist(labelsA))
    predsB<-as.numeric(unlist(predsB))
    labelsB<-as.numeric(unlist(labelsB))
    predsC<-as.numeric(unlist(predsC))
    labelsC<-as.numeric(unlist(labelsC))

    library(pROC)
    
    roc1 <- roc(labelsA, predsA, percent=FALSE, 
    smooth=TRUE, ci=TRUE, boot.n=100, ci.alpha=0.95, 
    stratified=TRUE,plot=FALSE,grid=FALSE,print.auc=FALSE, show.thres=TRUE, col='red')
    roc2 <- roc(labelsB, predsB, percent=FALSE, 
    smooth=TRUE, ci=TRUE, boot.n=100, ci.alpha=0.95, 
    stratified=TRUE,plot=FALSE,grid=FALSE,print.auc=FALSE, show.thres=TRUE, col='red')
    roc3 <- roc(labelsC, predsC, percent=FALSE, 
    smooth=TRUE, ci=TRUE, boot.n=100, ci.alpha=0.95, 
    stratified=TRUE,plot=FALSE,grid=FALSE,print.auc=FALSE, show.thres=TRUE, col='red')
    roc_test12<-roc.test(roc1,roc2)
    roc_test13<-roc.test(roc1,roc3)
    test_accuracy1<-coords(roc1, "best", ret=c("specificity","sensitivity","accuracy"), best.method=c("youden","closest.topleft"));
    test_accuracy2<-coords(roc2, "best", ret=c("specificity","sensitivity","accuracy"), best.method=c("youden","closest.topleft"));
    test_accuracy3<-coords(roc3, "best", ret=c("specificity","sensitivity","accuracy"), best.method=c("youden","closest.topleft"));
''')

# Generate final form of Table 4 
columns=['Model','AUC','Accuracy','Sensitivity','Specificity','p-value'
        ]
results = pd.DataFrame(
    {'Model':[],
     'AUC':[],
     'Accuracy':[],
     'Sensitivity':[],
     'Specificity':[],
     'p-value':[]
             },
    columns=columns
)

results = results.append(
    pd.DataFrame(
        [
            [
                'Model 1',
                robjects.r["roc1"].rx2('auc')[0],
                robjects.r["test_accuracy1"][2],
                robjects.r["test_accuracy1"][1],
                robjects.r["test_accuracy1"][0],
                np.nan
            ],
            [
                'Model 2',
                robjects.r["roc2"].rx2('auc')[0],
                robjects.r["test_accuracy2"][2],
                robjects.r["test_accuracy2"][1],
                robjects.r["test_accuracy2"][0],
                robjects.r["roc_test12"].rx2('p.value')[0]
            ],
            [
                'Model 3',
                robjects.r["roc3"].rx2('auc')[0],
                robjects.r["test_accuracy3"][2],
                robjects.r["test_accuracy3"][1],
                robjects.r["test_accuracy3"][0],
                robjects.r["roc_test13"].rx2('p.value')[0]
            ]
        ],
        columns=columns
    ),
    ignore_index=True
)
cm = sns.light_palette("green",as_cmap=True)
results.style.background_gradient(cmap=cm,high=1.0,low=0.0)

<h1>Prediction of poor surgical outcome: Engel I vs. II-IV</h1>

<h2>Feature selection</h2>

In [ ]:
#Feature Selection for Model B

# Parameters for feature selection
n_repeats = 100
num_cv = 5

# Load data
df = pd.read_csv('../data/qPET_feature_matrix_clinical_3dssp.csv')
df_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_3dssp.csv')

# Generate feature matrix and target vectors
X = np.array(df[df.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X_labels = np.array(df.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))

# Generate output labels
y = np.array(df.outcomeLatest)
y[y<5] = 0
y[y>=5] = 1


all_features = []
for iter_id in range(n_repeats):
    X_resample, y_resample = sklearn.utils.resample(X,y,n_samples=int(0.7*X.shape[0]),replace=False, random_state=RANDOM_STATE)
    fdr = SelectKBest(f_classif,k=int(n_repeats/2))
    fdr.fit(X_resample, y_resample)
    for index in fdr.get_support(indices=True):
        all_features.append(index)
all_features =  np.sort(all_features)

ft_counts = {}
for ft in all_features:
    try:
        ft_counts[ft] += 1
    except KeyError:
        ft_counts[ft] = 1
SSP_best_features1 = []
for ft,ft_count in ft_counts.items():
    if ft_count > n_repeats/2:
        SSP_best_features1.append(ft)

estimator = RandomForestClassifier(n_estimators=100, max_depth=max_depth, random_state=RANDOM_STATE)
selector = RFECV(estimator, step=1, cv=num_cv)
selector = selector.fit(X,y)
SSP_best_features2 = np.where(selector.support_)[0]

SSP_features = list(set(SSP_best_features1) & set(SSP_best_features2))
print sorted(SSP_features)


In [ ]:
#Feature Selection for Model C

# Load data
df = pd.read_csv('../data/qPET_feature_matrix_clinical_voxel_ai.csv')
df_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_voxel_ai.csv')

# Generate feature matrix and target vectors
X = np.array(df[df.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X_labels = np.array(df.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
# Generate output labels
y = np.array(df.outcomeLatest)
y[y<5] = 0
y[y>=5] = 1

all_features = []
for iter_id in range(n_repeats):
    X_resample, y_resample = sklearn.utils.resample(X,y,n_samples=int(0.7*X.shape[0]),replace=False, random_state=RANDOM_STATE)
    fdr = SelectKBest(f_classif,k=int(n_repeats/2))
    fdr.fit(X_resample, y_resample)
    for index in fdr.get_support(indices=True):
        all_features.append(index)
all_features =  np.sort(all_features)

ft_counts = {}
for ft in all_features:
    try:
        ft_counts[ft] += 1
    except KeyError:
        ft_counts[ft] = 1
ASYMM_best_features1 = []
for ft,ft_count in ft_counts.items():
    if ft_count > n_repeats/2:
        ASYMM_best_features1.append(ft)

estimator = RandomForestClassifier(n_estimators=100, max_depth=max_depth, random_state=RANDOM_STATE)
selector = RFECV(estimator, step=1, cv=num_cv)
selector = selector.fit(X,y)
ASYMM_best_features2 = np.where(selector.support_)[0]

ASYMM_features = list(set(ASYMM_best_features1) & set(ASYMM_best_features2))
print sorted(ASYMM_features) 

In [7]:
SSP_features = [0, 4, 6, 14, 15, 16, 18, 19, 20, 21, 22, 28, 29, 32, 36, 37, 43, 51, 52, 59, 60, 62, 78, 81, 82, 84, 85, 96, 97, 98, 105, 108, 112, 113, 119, 127, 128, 135, 136, 138, 140, 141, 142, 143]
ASYMM_features = [43, 235]

In [ ]:
len(SSP_features), len(ASYMM_features)

<h2>Determine number of estimators</h2>

In [ ]:
# Load data for CLINICAL ONLY
dfA = pd.read_csv('../data/qPET_feature_matrix_clinical_only.csv')
dfA_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_only.csv')

# Generate feature matrix for training and testing data
# Training
X1 = np.array(dfA[dfA.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X1_labels = np.array(dfA.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))

# Load data for SSP ONLY
dfB = pd.read_csv('../data/qPET_feature_matrix_clinical_3dssp.csv')
dfB_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_3dssp.csv')

# Generate feature matrix for training and testing data
# Training
X2 = np.array(dfB[dfB.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X2_labels = np.array(dfB.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X2 = X2[:,SSP_features]
X2_labels = X2_labels[SSP_features]

# Load data for ASYMMETRY ONLY
dfC = pd.read_csv('../data/qPET_feature_matrix_clinical_voxel_ai.csv')
dfC_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_voxel_ai.csv')

# Generate feature matrix for training testing data
# Training
X3 = np.array(dfC[dfC.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X3_labels = np.array(dfC.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X3 = X3[:,ASYMM_features]
X3_labels = X3_labels[ASYMM_features]

# Generate outcome variable
outcome_threshold = 5 # i.e. Engel 1B
y = np.array(dfA.outcomeLatest)
y_test = np.array(dfA_test.outcomeLatest)
y[y<outcome_threshold] = 0
y[y>=outcome_threshold] = 1
y_test[y_test<outcome_threshold] = 0
y_test[y_test>=outcome_threshold] = 1


XA = X1
XA_labels = X1_labels
yA = y

XB = np.hstack((X1,X2))
XB_labels = np.hstack((X1_labels,X2_labels))
yB = y

yC = y
XC = np.hstack((X1,X3))
XC_labels = np.hstack((X1_labels,X3_labels))


min_estimators = 15
max_estimators = 1000

clfA = RandomForestClassifier(warm_start=True, max_features=None, 
                             oob_score=True, max_depth=max_depth, 
                             random_state=RANDOM_STATE)
error_rate = []
for i in range(min_estimators, max_estimators+1):
    clfA.set_params(n_estimators=i)
    clfA.fit(XA,yA)
    oob_error = 1-clfA.oob_score_
    error_rate.append((i, oob_error))
error_rate = np.array(error_rate)
plt.plot(error_rate[:,0], error_rate[:,1])
plt.xlim(min_estimators, max_estimators)
plt.title('OOB Error versus Number of Estimators for Clinical Only')
plt.xlabel('n_estimators')
plt.ylabel('OOB error rate')
plt.show()
nest = error_rate[:,0] 
err = error_rate[:,1]
n_estimatorsA = int(nest[np.where(err == np.min(err))[0][0]])

clfB = RandomForestClassifier(warm_start=True, max_features=None, 
                             oob_score=True, max_depth=max_depth, 
                             random_state=RANDOM_STATE)
error_rate = []
for i in range(min_estimators, max_estimators+1):
    clfB.set_params(n_estimators=i)
    clfB.fit(XB,yB)
    oob_error = 1-clfB.oob_score_
    error_rate.append((i, oob_error))
error_rate = np.array(error_rate)
plt.plot(error_rate[:,0], error_rate[:,1])
plt.xlim(min_estimators, max_estimators)
plt.title('OOB Error versus Number of Estimators for Clinical Only')
plt.xlabel('n_estimators')
plt.ylabel('OOB error rate')
plt.show()
nest = error_rate[:,0] 
err = error_rate[:,1]
n_estimatorsB = int(nest[np.where(err == np.min(err))[0][0]])

clfC = RandomForestClassifier(warm_start=True, max_features=None, 
                             oob_score=True, max_depth=max_depth, 
                             random_state=RANDOM_STATE)
error_rate = []
for i in range(min_estimators, max_estimators+1):
    clfC.set_params(n_estimators=i)
    clfC.fit(XC,yC)
    oob_error = 1-clfC.oob_score_
    error_rate.append((i, oob_error))
error_rate = np.array(error_rate)
plt.plot(error_rate[:,0], error_rate[:,1])
plt.xlim(min_estimators, max_estimators)
plt.title('OOB Error versus Number of Estimators for Clinical Only')
plt.xlabel('n_estimators')
plt.ylabel('OOB error rate')
plt.show()
nest = error_rate[:,0] 
err = error_rate[:,1]
n_estimatorsC = int(nest[np.where(err == np.min(err))[0][0]])

<h2>Measure cross-validation scores</h2>

This section ...

In [ ]:
# Preprocessing
classifierA = RandomForestClassifier(n_estimators=n_estimatorsA, max_depth=max_depth, random_state=RANDOM_STATE,oob_score=True)
classifierB = RandomForestClassifier(n_estimators=n_estimatorsB, max_depth=max_depth, random_state=RANDOM_STATE, oob_score=True)
classifierC = RandomForestClassifier(n_estimators=n_estimatorsC, max_depth=max_depth, random_state=RANDOM_STATE, oob_score=True)

sss = StratifiedShuffleSplit(n_splits=NUM_BOOTSTRAPS, test_size=0.2, random_state=RANDOM_STATE)

print 'Generating feature matrices ...'
# Load data for CLINICAL ONLY
dfA = pd.read_csv('../data/qPET_feature_matrix_clinical_only.csv')
dfA_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_only.csv')

# Generate feature matrix for training and testing data
# Training
X1 = np.array(dfA[dfA.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X1_labels = np.array(dfA.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
# Testing
X1_test = np.array(dfA_test[dfA_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X1_test_labels = np.array(dfA_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))

# Load data for SSP ONLY
dfB = pd.read_csv('../data/qPET_feature_matrix_clinical_3dssp.csv')
dfB_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_3dssp.csv')

# Generate feature matrix for training and testing data
# Training
X2 = np.array(dfB[dfB.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X2_labels = np.array(dfB.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X2 = X2[:,SSP_features]
X2_labels = X2_labels[SSP_features]
# Testing
X2_test = np.array(dfB_test[dfB_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X2_test_labels = np.array(dfB_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X2_test = X2_test[:,SSP_features]
X2_test_labels = X2_test_labels[SSP_features]

# Load data for ASYMMETRY ONLY
dfC = pd.read_csv('../data/qPET_feature_matrix_clinical_voxel_ai.csv')
dfC_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_voxel_ai.csv')

# Generate feature matrix for training testing data
# Training
X3 = np.array(dfC[dfC.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X3_labels = np.array(dfC.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X3 = X3[:,ASYMM_features]
X3_labels = X3_labels[ASYMM_features]
# Testing
X3_test = np.array(dfC_test[dfC_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X3_test_labels = np.array(dfC_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X3_test = X3_test[:,ASYMM_features]
X3_test_labels = X3_test_labels[ASYMM_features]

# Generate outcome variable
outcome_threshold = 5 # i.e. Engel 1B
y = np.array(dfA.outcomeLatest)
y_test = np.array(dfA_test.outcomeLatest)
y[y<outcome_threshold] = 0
y[y>=outcome_threshold] = 1
y_test[y_test<outcome_threshold] = 0
y_test[y_test>=outcome_threshold] = 1

In [ ]:
columns=['Clinical Variable(s)','CV 5-Fold AUC_1','Out Of Bag (OOB1) Error',
         'Quantitative Variables','CV 5-Fold AUC_2','Out Of Bag (OOB2) Error',
         'AUC Difference 95% C.I.','OOB Difference 95% C.I.'
        ]
results = pd.DataFrame(
    {'Clinical Variable(s)':[],
     'CV 5-Fold AUC_1':[],
     'Out Of Bag (OOB1) Error':[],
     'Quantitative Variables':[],
     'CV 5-Fold AUC_2':[],
     'Out Of Bag (OOB2) Error':[],
     'AUC Difference 95% C.I.':[],
     'OOB Difference 95% C.I.':[]
             },
    columns=columns
)

OPTIONS = {
    1:[np.arange(20),'EEG+MRI+PET'],
    2:[np.arange(12),'EEG'],
    3:[np.arange(12,14),'MRI'],
    4:[np.arange(13,20),'PET'],
}

for OPTION,(clinical_variable_idx,variables_list) in OPTIONS.items():
    # Generate feature matrix for CLINICAL ONLY
    XA = X1[:,clinical_variable_idx]
    XA_labels = X1_labels[clinical_variable_idx]
    yA = y

    # Generate feature matrix for SSP alone
    XB = X2
    XB_labels = X2_labels
    XB_test = X2_test
    XB_test_labels = X2_test_labels
    yB = y

    # Generate bootstrap jobs
    jobs = []
    out = []
    for job_iter,(train, test) in enumerate(sss.split(XA,yA)):
        jobs.append((job_iter,train,test,XA,yA,XB,yB,classifierA,classifierB))
    # Run all jobs
    return_list = pool.map(_helper,jobs)
    # Compute OOB/AUC for bootstraps
    for res in return_list:
        probas_A,yA_test,probas_B,yB_test,classifierA_oob_score_,classifierB_oob_score_ = res
        fprA, tprA, thresholds = roc_curve(yA_test, probas_A[:, 1])
        fprB, tprB, thresholds = roc_curve(yB_test, probas_B[:, 1])
        roc_aucA = auc(fprA, tprA)
        roc_aucB = auc(fprB, tprB)
        out.append([roc_aucA,roc_aucB,classifierA_oob_score_,classifierB_oob_score_])
    out21 = np.array(out)

    # Generate feature matrix for SSP and CLINICAL
    XB = np.hstack((X1,X2))
    XB_labels = np.hstack((X1_labels,X2_labels))
    XB_test = np.hstack((X1_test,X2_test))
    XB_test_labels = np.hstack((X1_test_labels,X2_test_labels))

    # Generate bootstrap jobs
    jobs = []
    out = []
    for job_iter,(train, test) in enumerate(sss.split(XA,yA)):
        jobs.append((job_iter,train,test,XA,yA,XB,yB,classifierA,classifierB))
    # Run all jobs
    return_list = pool.map(_helper,jobs)
    # Compute OOB/AUC for bootstraps
    for res in return_list:
        probas_A,yA_test,probas_B,yB_test,classifierA_oob_score_,classifierB_oob_score_ = res
        fprA, tprA, thresholds = roc_curve(yA_test, probas_A[:, 1])
        fprB, tprB, thresholds = roc_curve(yB_test, probas_B[:, 1])
        roc_aucA = auc(fprA, tprA)
        roc_aucB = auc(fprB, tprB)
        out.append([roc_aucA,roc_aucB,classifierA_oob_score_,classifierB_oob_score_])
    out31 = np.array(out)

    # Compute statistics
    AUC1 = np.mean(out21[:,0])
    OOB1 = np.mean(out21[:,2])
    AUC2 = np.mean(out21[:,1])
    OOB2 = np.mean(out21[:,3])
    AUC3 = np.mean(out31[:,1])
    OOB3 = np.mean(out31[:,3])
    CI_AUC2_minus_AUC1 = tuple(map(lambda x: np.percentile(out21[:,1] - out21[:,0],x), [2.5, 97.5]))
    CI_AUC3_minus_AUC1 = tuple(map(lambda x: np.percentile(out31[:,1] - out31[:,0],x), [2.5, 97.5]))
    CI_OOB2_minus_OOB1 = tuple(map(lambda x: np.percentile(out21[:,3] - out21[:,2],x), [2.5, 97.5]))
    CI_OOB3_minus_OOB1 = tuple(map(lambda x: np.percentile(out31[:,3] - out31[:,2],x), [2.5, 97.5]))


    results = results.append(
        pd.DataFrame(
            [
                [variables_list,AUC1,OOB1,'%s+SSP'%(variables_list), AUC2, OOB2, CI_AUC2_minus_AUC1, CI_OOB2_minus_OOB1],
                [variables_list,AUC1,OOB1,'SSP only', AUC3, OOB3, CI_AUC3_minus_AUC1, CI_OOB3_minus_OOB1]
            ],
            columns=columns
        ),
        ignore_index=True
    )
cm = sns.light_palette("green",as_cmap=True)
results.style.background_gradient(cmap=cm,high=1.0)

In [ ]:
columns=['Clinical Variable(s)','CV 5-Fold AUC_1','Out Of Bag (OOB1) Error',
         'Quantitative Variables','CV 5-Fold AUC_2','Out Of Bag (OOB2) Error',
         'AUC Difference 95% C.I.','OOB Difference 95% C.I.'
        ]
results = pd.DataFrame(
    {'Clinical Variable(s)':[],
     'CV 5-Fold AUC_1':[],
     'Out Of Bag (OOB1) Error':[],
     'Quantitative Variables':[],
     'CV 5-Fold AUC_2':[],
     'Out Of Bag (OOB2) Error':[],
     'AUC Difference 95% C.I.':[],
     'OOB Difference 95% C.I.':[]
             },
    columns=columns
)

OPTIONS = {
    1:[np.arange(20),'EEG+MRI+PET'],
    2:[np.arange(12),'EEG'],
    3:[np.arange(12,14),'MRI'],
    4:[np.arange(13,20),'PET'],
}

for OPTION,(clinical_variable_idx,variables_list) in OPTIONS.items():
    # Generate feature matrix for CLINICAL ONLY
    XA = X1[:,clinical_variable_idx]
    XA_labels = X1_labels[clinical_variable_idx]
    yA = y

    # Generate feature matrix for SSP alone
    XC = X3
    XC_labels = X3_labels
    XC_test = X3_test
    XC_test_labels = X3_test_labels
    yC = y

    # Generate bootstrap jobs
    jobs = []
    out = []
    for job_iter,(train, test) in enumerate(sss.split(XA,yA)):
        jobs.append((job_iter,train,test,XA,yA,XC,yC,classifierA,classifierC))
    # Run all jobs
    return_list = pool.map(_helper,jobs)
    # Compute OOB/AUC for bootstraps
    for res in return_list:
        probas_A,yA_test,probas_C,yC_test,classifierA_oob_score_,classifierC_oob_score_ = res
        fprA, tprA, thresholds = roc_curve(yA_test, probas_A[:, 1])
        fprC, tprC, thresholds = roc_curve(yC_test, probas_C[:, 1])
        roc_aucA = auc(fprA, tprA)
        roc_aucC = auc(fprC, tprC)
        out.append([roc_aucA,roc_aucC,classifierA_oob_score_,classifierC_oob_score_])
    out21 = np.array(out)

    # Generate feature matrix for SSP and CLINICAL
    XC = np.hstack((X1,X3))
    XC_labels = np.hstack((X1_labels,X3_labels))
    XC_test = np.hstack((X1_test,X3_test))
    XC_test_labels = np.hstack((X1_test_labels,X3_test_labels))

    # Generate bootstrap jobs
    jobs = []
    out = []
    for job_iter,(train, test) in enumerate(sss.split(XA,yA)):
        jobs.append((job_iter,train,test,XA,yA,XC,yB,classifierA,classifierB))
    # Run all jobs
    return_list = pool.map(_helper,jobs)
    # Compute OOB/AUC for bootstraps
    for res in return_list:
        probas_A,yA_test,probas_C,yC_test,classifierA_oob_score_,classifierC_oob_score_ = res
        fprA, tprA, thresholds = roc_curve(yA_test, probas_A[:, 1])
        fprC, tprC, thresholds = roc_curve(yC_test, probas_C[:, 1])
        roc_aucA = auc(fprA, tprA)
        roc_aucC = auc(fprC, tprC)
        out.append([roc_aucA,roc_aucC,classifierA_oob_score_,classifierC_oob_score_])
    out31 = np.array(out)

    # Compute statistics
    AUC1 = np.mean(out21[:,0])
    OOB1 = np.mean(out21[:,2])
    AUC2 = np.mean(out21[:,1])
    OOB2 = np.mean(out21[:,3])
    AUC3 = np.mean(out31[:,1])
    OOB3 = np.mean(out31[:,3])
    CI_AUC2_minus_AUC1 = tuple(map(lambda x: np.percentile(out21[:,1] - out21[:,0],x), [2.5, 97.5]))
    CI_AUC3_minus_AUC1 = tuple(map(lambda x: np.percentile(out31[:,1] - out31[:,0],x), [2.5, 97.5]))
    CI_OOB2_minus_OOB1 = tuple(map(lambda x: np.percentile(out21[:,3] - out21[:,2],x), [2.5, 97.5]))
    CI_OOB3_minus_OOB1 = tuple(map(lambda x: np.percentile(out31[:,3] - out31[:,2],x), [2.5, 97.5]))


    results = results.append(
        pd.DataFrame(
            [
                [variables_list,AUC1,OOB1,'%s+Asymmetry'%(variables_list), AUC2, OOB2, CI_AUC2_minus_AUC1, CI_OOB2_minus_OOB1],
                [variables_list,AUC1,OOB1,'Asymmetry only', AUC3, OOB3, CI_AUC3_minus_AUC1, CI_OOB3_minus_OOB1]
            ],
            columns=columns
        ),
        ignore_index=True
    )
cm = sns.light_palette("green",as_cmap=True)
results.style.background_gradient(cmap=cm,high=1.0)

<h2>Feature importances (Table 3)</h2>

In [8]:
# Create feature matrix for training and testing
# Load data for CLINICAL ONLY
dfA = pd.read_csv('../data/qPET_feature_matrix_clinical_only.csv')
dfA_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_only.csv')

# Generate feature matrix for training and testing data
# Training
X1 = np.array(dfA[dfA.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X1_labels = np.array(dfA.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
# Testing
X1_test = np.array(dfA_test[dfA_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X1_test_labels = np.array(dfA_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))

# Load data for SSP ONLY
dfB = pd.read_csv('../data/qPET_feature_matrix_clinical_3dssp.csv')
dfB_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_3dssp.csv')

# Generate feature matrix for training and testing data
# Training
X2 = np.array(dfB[dfB.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X2_labels = np.array(dfB.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X2 = X2[:,SSP_features]
X2_labels = X2_labels[SSP_features]
# Testing
X2_test = np.array(dfB_test[dfB_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X2_test_labels = np.array(dfB_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X2_test = X2_test[:,SSP_features]
X2_test_labels = X2_test_labels[SSP_features]

# Load data for ASYMMETRY ONLY
dfC = pd.read_csv('../data/qPET_feature_matrix_clinical_voxel_ai.csv')
dfC_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_voxel_ai.csv')

# Generate feature matrix for training testing data
# Training
X3 = np.array(dfC[dfC.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X3_labels = np.array(dfC.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X3 = X3[:,ASYMM_features]
X3_labels = X3_labels[ASYMM_features]
# Testing
X3_test = np.array(dfC_test[dfC_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X3_test_labels = np.array(dfC_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X3_test = X3_test[:,ASYMM_features]
X3_test_labels = X3_test_labels[ASYMM_features]

# Generate outcome variable
outcome_threshold = 5 # i.e. Engel 1B
y = np.array(dfA.outcomeLatest)
y_test = np.array(dfA_test.outcomeLatest)
# y[y<outcome_threshold] = 0
# y[y>=outcome_threshold] = 1
# y_test[y_test<outcome_threshold] = 0
# y_test[y_test>=outcome_threshold] = 1

# Run predictions
# Generate feature matrix for CLINICAL ONLY
XA = X1
XA_labels = X1_labels
XA_test = X1_test
XA_test_labels = X1_test_labels
yA = y

# Generate feature matrix for SSP and CLINICAL
XB = np.hstack((X1,X2))
XB_labels = np.hstack((X1_labels,X2_labels))
XB_test = np.hstack((X1_test,X2_test))
XB_test_labels = np.hstack((X1_test_labels,X2_test_labels))
yB = y

# Generate feature matrix for SSP and CLINICAL
XC = np.hstack((X1,X3))
XC_labels = np.hstack((X1_labels,X3_labels))
XC_test = np.hstack((X1_test,X3_test))
XC_test_labels = np.hstack((X1_test_labels,X3_test_labels))
yC = y


In [9]:
robjects.globalenv['RANDOM_STATE'] = RANDOM_STATE

# Generate final form of Table 2
columns=[
    'Feature',
    'Feature Importance',
    'Univariate Odds Ratio per Unit\n Increase in Seizure Recurrence [95% CI, p]'
        ]
results = pd.DataFrame(
    {'Feature':[],
     'Feature Importance':[],
     'Univariate Odds Ratio per Unit\n Increase in Seizure Recurrence [95% CI, p]':[]
             },
    columns=columns
)

for X,y, X_labels in [(XA,yA,XA_labels), (XB,yB,XB_labels), (XC,yC,XC_labels)]:
    # Run feature importance (IncMSE) for random forests
    robjects.globalenv['X'] = X
    robjects.globalenv['y'] = y
    summary = robjects.r['summary']

    robjects.r('''
        set.seed(RANDOM_STATE)
        library(randomForest)        
        rf <- randomForest(y ~ ., data=X, importance=TRUE)
        X_importances <- importance(rf)    
    ''')
    X_importances = robjects.r['X_importances']
    feature_order = np.argsort(-X_importances[:,0])

    # Run univariate odds ratio computation
    for feature in feature_order:
        robjects.globalenv['feature'] = X[:,feature]    
        robjects.r('''        
            set.seed(RANDOM_STATE)
            require(MASS)        
            myfit <- polr(as.factor(y) ~ feature, Hess=TRUE)
            (ctable <- coef(summary(myfit)))            
            p <- pnorm(abs(ctable[,"t value"]), lower.tail=FALSE)*2
            (ctable <- cbind(ctable, "p value" = p))
            (ci <- confint(myfit))
            myfit_ci <-exp(cbind(OR = coef(myfit), ci))            
        ''')
        myfit = robjects.r['myfit']
        myfit_ci = robjects.r['myfit_ci']
        pval = robjects.r['p'][0]
        label = X_labels[feature]
        try:
            label = label.replace(label.split('_')[-1],ssp_labels[int(label.split('_')[-1])])
        except Exception:
            pass

        if X_importances[feature,0] <= 0.0:
            continue
        
        if pval < 0.001:
            pval = '%0.3f ***'%pval
        elif pval < 0.01:
            pval = '%0.3f **'%pval
        elif pval < 0.05:
            pval = '%0.3f *'%pval
        elif pval < 0.1:
            pval = '%0.3f .'%pval
        else:
            pval = '%0.2f'%pval
        
        results = results.append(
            pd.DataFrame(
                [
                    [
                        label,
                        '%0.2f'%X_importances[feature,0], 
                        '%0.2f [%0.2f - %0.2f, %s]'%(
                            myfit_ci[0,0],
                            myfit_ci[0,1], myfit_ci[1,1], 
                            pval
                        )
                    ]
                ],
                columns=columns
            ),
            ignore_index=True
        )
    results = results.append(
        pd.DataFrame(
            [
                [
                    '',
                    '', 
                    ''
                ]
            ],
            columns=columns
        ),
        ignore_index=True
    )
# cm = sns.light_palette("green",as_cmap=True)
# results.style.background_gradient(cmap=cm,high=1.0,low=0.0)
display(results)
results.to_csv('results_Section4.csv')

,Feature,Feature Importance,"Univariate Odds Ratio per Unit Increase in Seizure Recurrence [95% CI, p]"
0,petclass_S,10.04,"0.29 [0.10 - 0.77, 0.016 *]"
1,eeg_D,5.28,"3.56 [0.76 - 17.79, 0.11]"
2,eeg_LTL+,4.89,"3.00 [0.72 - 12.90, 0.13]"
3,eeg_RTL+,4.82,"0.40 [0.05 - 2.59, 0.33]"
4,petclass_M,3.61,"3.36 [1.16 - 10.00, 0.026 *]"
5,petclass_R,2.02,"0.71 [0.32 - 1.60, 0.41]"
6,lesional_NL,1.08,"2.35 [1.04 - 5.39, 0.041 *]"
7,,,
8,contra_z_ai_3dssp_Angular Gyrus,5.55,"1.89 [1.22 - 3.02, 0.006 **]"
9,contra_z_3dssp_Angular Gyrus,5.27,"1.03 [0.86 - 1.26, 0.79]"


<h2>Validation on testing set (Table 4)</h2>

In [ ]:
# Create feature matrix for training and testing
# Load data for CLINICAL ONLY
dfA = pd.read_csv('../data/qPET_feature_matrix_clinical_only.csv')
dfA_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_only.csv')

# Generate feature matrix for training and testing data
# Training
X1 = np.array(dfA[dfA.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X1_labels = np.array(dfA.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
# Testing
X1_test = np.array(dfA_test[dfA_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X1_test_labels = np.array(dfA_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))

# Load data for SSP ONLY
dfB = pd.read_csv('../data/qPET_feature_matrix_clinical_3dssp.csv')
dfB_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_3dssp.csv')

# Generate feature matrix for training and testing data
# Training
X2 = np.array(dfB[dfB.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X2_labels = np.array(dfB.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X2 = X2[:,SSP_features]
X2_labels = X2_labels[SSP_features]
# Testing
X2_test = np.array(dfB_test[dfB_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X2_test_labels = np.array(dfB_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X2_test = X2_test[:,SSP_features]
X2_test_labels = X2_test_labels[SSP_features]

# Load data for ASYMMETRY ONLY
dfC = pd.read_csv('../data/qPET_feature_matrix_clinical_voxel_ai.csv')
dfC_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_voxel_ai.csv')

# Generate feature matrix for training testing data
# Training
X3 = np.array(dfC[dfC.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X3_labels = np.array(dfC.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X3 = X3[:,ASYMM_features]
X3_labels = X3_labels[ASYMM_features]
# Testing
X3_test = np.array(dfC_test[dfC_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X3_test_labels = np.array(dfC_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X3_test = X3_test[:,ASYMM_features]
X3_test_labels = X3_test_labels[ASYMM_features]

# Generate outcome variable
outcome_threshold = 5 # i.e. Engel 1B
y = np.array(dfA.outcomeLatest)
y_test = np.array(dfA_test.outcomeLatest)
y[y<outcome_threshold] = 0
y[y>=outcome_threshold] = 1
y_test[y_test<outcome_threshold] = 0
y_test[y_test>=outcome_threshold] = 1

# Run predictions
# Generate feature matrix for CLINICAL ONLY
XA = X1
XA_labels = X1_labels
XA_test = X1_test
XA_test_labels = X1_test_labels

# Generate feature matrix for SSP and CLINICAL
XB = np.hstack((X1,X2))
XB_labels = np.hstack((X1_labels,X2_labels))
XB_test = np.hstack((X1_test,X2_test))
XB_test_labels = np.hstack((X1_test_labels,X2_test_labels))

# Generate feature matrix for SSP and CLINICAL
XC = np.hstack((X1,X3))
XC_labels = np.hstack((X1_labels,X3_labels))
XC_test = np.hstack((X1_test,X3_test))
XC_test_labels = np.hstack((X1_test_labels,X3_test_labels))

# Load library in R
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
pROC = importr('pROC')

# Train classifier and apply to validation test set
classifierA = RandomForestClassifier(
    n_estimators=n_estimatorsA, 
    max_depth=max_depth, 
    random_state=RANDOM_STATE,
    oob_score=True)
classifierB = RandomForestClassifier(
    n_estimators=n_estimatorsB, 
    max_depth=max_depth, 
    random_state=RANDOM_STATE,
    oob_score=True)
classifierC = RandomForestClassifier(
    n_estimators=n_estimatorsC, 
    max_depth=max_depth, 
    random_state=RANDOM_STATE,
    oob_score=True)

probas = classifierA.fit(XA, y).predict_proba(XA_test)
predsA = robjects.FloatVector(probas[:,1])
labelsA = robjects.IntVector(y_test)
probas = classifierB.fit(XB, y).predict_proba(XB_test)
predsB = robjects.FloatVector(probas[:,1])
labelsB = robjects.IntVector(y_test)
probas = classifierC.fit(XC, y).predict_proba(XC_test)
predsC = robjects.FloatVector(probas[:,1])
labelsC = robjects.IntVector(y_test)


# Copy from python workspace to R workspace
robjects.globalenv["predsA"] = predsA
robjects.globalenv["labelsA"] = labelsA
robjects.globalenv["predsB"] = predsB
robjects.globalenv["labelsB"] = labelsB
robjects.globalenv["predsC"] = predsC
robjects.globalenv["labelsC"] = labelsC

# Run pROC.roc and pROC.roc.test in R
robjects.r('''
    predsA<-as.numeric(unlist(predsA))
    labelsA<-as.numeric(unlist(labelsA))
    predsB<-as.numeric(unlist(predsB))
    labelsB<-as.numeric(unlist(labelsB))
    predsC<-as.numeric(unlist(predsC))
    labelsC<-as.numeric(unlist(labelsC))

    library(pROC)
    
    roc1 <- roc(labelsA, predsA, percent=FALSE, 
    smooth=TRUE, ci=TRUE, boot.n=100, ci.alpha=0.95, 
    stratified=TRUE,plot=FALSE,grid=FALSE,print.auc=FALSE, show.thres=TRUE, col='red')
    roc2 <- roc(labelsB, predsB, percent=FALSE, 
    smooth=TRUE, ci=TRUE, boot.n=100, ci.alpha=0.95, 
    stratified=TRUE,plot=FALSE,grid=FALSE,print.auc=FALSE, show.thres=TRUE, col='red')
    roc3 <- roc(labelsC, predsC, percent=FALSE, 
    smooth=TRUE, ci=TRUE, boot.n=100, ci.alpha=0.95, 
    stratified=TRUE,plot=FALSE,grid=FALSE,print.auc=FALSE, show.thres=TRUE, col='red')
    roc_test12<-roc.test(roc1,roc2)
    roc_test13<-roc.test(roc1,roc3)
    test_accuracy1<-coords(roc1, "best", ret=c("specificity","sensitivity","accuracy"), best.method=c("youden","closest.topleft"));
    test_accuracy2<-coords(roc2, "best", ret=c("specificity","sensitivity","accuracy"), best.method=c("youden","closest.topleft"));
    test_accuracy3<-coords(roc3, "best", ret=c("specificity","sensitivity","accuracy"), best.method=c("youden","closest.topleft"));
''')

# Generate final form of Table 4 
columns=['Model','AUC','Accuracy','Sensitivity','Specificity','p-value'
        ]
results = pd.DataFrame(
    {'Model':[],
     'AUC':[],
     'Accuracy':[],
     'Sensitivity':[],
     'Specificity':[],
     'p-value':[]
             },
    columns=columns
)

results = results.append(
    pd.DataFrame(
        [
            [
                'Model 1',
                robjects.r["roc1"].rx2('auc')[0],
                robjects.r["test_accuracy1"][2],
                robjects.r["test_accuracy1"][1],
                robjects.r["test_accuracy1"][0],
                np.nan
            ],
            [
                'Model 2',
                robjects.r["roc2"].rx2('auc')[0],
                robjects.r["test_accuracy2"][2],
                robjects.r["test_accuracy2"][1],
                robjects.r["test_accuracy2"][0],
                robjects.r["roc_test12"].rx2('p.value')[0]
            ],
            [
                'Model 3',
                robjects.r["roc3"].rx2('auc')[0],
                robjects.r["test_accuracy3"][2],
                robjects.r["test_accuracy3"][1],
                robjects.r["test_accuracy3"][0],
                robjects.r["roc_test13"].rx2('p.value')[0]
            ]
        ],
        columns=columns
    ),
    ignore_index=True
)
cm = sns.light_palette("green",as_cmap=True)
results.style.background_gradient(cmap=cm,high=1.0,low=0.0)

<h1>Prediction of seizure recurrence in Engel I: Engel IA vs. IB-D</h1>

<h2>Feature selection</h2>

In [ ]:
#Feature Selection for Model B

# Parameters for feature selection
n_repeats = 100
num_cv = 5

# Load data
df = pd.read_csv('../data/qPET_feature_matrix_clinical_3dssp.csv')
df_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_3dssp.csv')
drop_idx = np.where(np.array(df.outcomeLatest) > 4)[0]
df = df.drop(df.index[drop_idx])


# Generate feature matrix and target vectors
X = np.array(df[df.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X_labels = np.array(df.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))

# Generate output labels
y = np.array(df.outcomeLatest)
y[y<2] = 0
y[y>=2] = 1


all_features = []
for iter_id in range(n_repeats):
    X_resample, y_resample = sklearn.utils.resample(X,y,n_samples=int(0.7*X.shape[0]),replace=False, random_state=RANDOM_STATE)
    fdr = SelectKBest(f_classif,k=int(n_repeats/2))
    fdr.fit(X_resample, y_resample)
    for index in fdr.get_support(indices=True):
        all_features.append(index)
all_features =  np.sort(all_features)

ft_counts = {}
for ft in all_features:
    try:
        ft_counts[ft] += 1
    except KeyError:
        ft_counts[ft] = 1
SSP_best_features1 = []
for ft,ft_count in ft_counts.items():
    if ft_count > n_repeats/2:
        SSP_best_features1.append(ft)

estimator = RandomForestClassifier(n_estimators=100, max_depth=max_depth, random_state=RANDOM_STATE)
selector = RFECV(estimator, step=1, cv=num_cv)
selector = selector.fit(X,y)
SSP_best_features2 = np.where(selector.support_)[0]

SSP_features = list(set(SSP_best_features1) & set(SSP_best_features2))
print sorted(SSP_features)


In [ ]:
#Feature Selection for Model C

# Load data
df = pd.read_csv('../data/qPET_feature_matrix_clinical_voxel_ai.csv')
df_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_voxel_ai.csv')
drop_idx = np.where(np.array(df.outcomeLatest) > 4)[0]
df = df.drop(df.index[drop_idx])

# Generate feature matrix and target vectors
X = np.array(df[df.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X_labels = np.array(df.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
# Generate output labels
y = np.array(df.outcomeLatest)
y[y<2] = 0
y[y>=2] = 1

all_features = []
for iter_id in range(n_repeats):
    X_resample, y_resample = sklearn.utils.resample(X,y,n_samples=int(0.7*X.shape[0]),replace=False, random_state=RANDOM_STATE)
    fdr = SelectKBest(f_classif,k=int(n_repeats/2))
    fdr.fit(X_resample, y_resample)
    for index in fdr.get_support(indices=True):
        all_features.append(index)
all_features =  np.sort(all_features)

ft_counts = {}
for ft in all_features:
    try:
        ft_counts[ft] += 1
    except KeyError:
        ft_counts[ft] = 1
ASYMM_best_features1 = []
for ft,ft_count in ft_counts.items():
    if ft_count > n_repeats/2:
        ASYMM_best_features1.append(ft)

estimator = RandomForestClassifier(n_estimators=100, max_depth=max_depth, random_state=RANDOM_STATE)
selector = RFECV(estimator, step=1, cv=num_cv)
selector = selector.fit(X,y)
ASYMM_best_features2 = np.where(selector.support_)[0]

ASYMM_features = list(set(ASYMM_best_features1) & set(ASYMM_best_features2))
print sorted(ASYMM_features) 

In [ ]:
SSP_features = [1, 2, 5, 8, 9, 12, 19, 22, 29, 30, 32, 33, 37, 38, 39, 40, 41, 51, 52, 58, 60, 64, 68, 78, 80, 84, 85, 88, 91, 94, 98, 102, 104, 106, 108, 109, 113, 114, 115, 116, 117, 118, 127, 128, 134, 136, 140, 141]
ASYMM_features = [12, 25, 27, 34, 42, 54, 73, 75, 77, 79, 88, 217, 219, 226, 234, 246, 250, 263, 265, 267, 269, 271, 280, 299, 339, 352]

In [ ]:
len(SSP_features), len(ASYMM_features)

<h2>Determine number of estimators</h2>

In [ ]:
# Load data for CLINICAL ONLY
dfA = pd.read_csv('../data/qPET_feature_matrix_clinical_only.csv')
dfA_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_only.csv')
drop_idx = np.where(np.array(dfA.outcomeLatest) > 4)[0]
drop_test_idx = np.where(np.array(dfA_test.outcomeLatest) > 4)[0]
dfA = dfA.drop(dfA.index[drop_idx])
dfA_test = dfA_test.drop(dfA_test.index[drop_test_idx])

# Generate feature matrix for training and testing data
# Training
X1 = np.array(dfA[dfA.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X1_labels = np.array(dfA.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))

# Load data for SSP ONLY
dfB = pd.read_csv('../data/qPET_feature_matrix_clinical_3dssp.csv')
dfB_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_3dssp.csv')
drop_idx = np.where(np.array(dfB.outcomeLatest) > 4)[0]
drop_test_idx = np.where(np.array(dfB_test.outcomeLatest) > 4)[0]
dfB = dfB.drop(dfB.index[drop_idx])
dfB_test = dfB_test.drop(dfB_test.index[drop_test_idx])

# Generate feature matrix for training and testing data
# Training
X2 = np.array(dfB[dfB.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X2_labels = np.array(dfB.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X2 = X2[:,SSP_features]
X2_labels = X2_labels[SSP_features]

# Load data for ASYMMETRY ONLY
dfC = pd.read_csv('../data/qPET_feature_matrix_clinical_voxel_ai.csv')
dfC_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_voxel_ai.csv')
drop_idx = np.where(np.array(dfC.outcomeLatest) > 4)[0]
drop_test_idx = np.where(np.array(dfC_test.outcomeLatest) > 4)[0]
dfC = dfC.drop(dfC.index[drop_idx])
dfC_test = dfC_test.drop(dfC_test.index[drop_test_idx])

# Generate feature matrix for training testing data
# Training
X3 = np.array(dfC[dfC.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X3_labels = np.array(dfC.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X3 = X3[:,ASYMM_features]
X3_labels = X3_labels[ASYMM_features]

# Generate outcome variable
outcome_threshold = 2 # i.e. Engel 1B
y = np.array(dfA.outcomeLatest)
y_test = np.array(dfA_test.outcomeLatest)
y[y<outcome_threshold] = 0
y[y>=outcome_threshold] = 1
y_test[y_test<outcome_threshold] = 0
y_test[y_test>=outcome_threshold] = 1


XA = X1
XA_labels = X1_labels
yA = y

XB = np.hstack((X1,X2))
XB_labels = np.hstack((X1_labels,X2_labels))
yB = y

yC = y
XC = np.hstack((X1,X3))
XC_labels = np.hstack((X1_labels,X3_labels))


min_estimators = 15
max_estimators = 1000

clfA = RandomForestClassifier(warm_start=True, max_features=None, 
                             oob_score=True, max_depth=max_depth, 
                             random_state=RANDOM_STATE)
error_rate = []
for i in range(min_estimators, max_estimators+1):
    clfA.set_params(n_estimators=i)
    clfA.fit(XA,yA)
    oob_error = 1-clfA.oob_score_
    error_rate.append((i, oob_error))
error_rate = np.array(error_rate)
plt.plot(error_rate[:,0], error_rate[:,1])
plt.xlim(min_estimators, max_estimators)
plt.title('OOB Error versus Number of Estimators for Clinical Only')
plt.xlabel('n_estimators')
plt.ylabel('OOB error rate')
plt.show()
nest = error_rate[:,0] 
err = error_rate[:,1]
n_estimatorsA = int(nest[np.where(err == np.min(err))[0][0]])

clfB = RandomForestClassifier(warm_start=True, max_features=None, 
                             oob_score=True, max_depth=max_depth, 
                             random_state=RANDOM_STATE)
error_rate = []
for i in range(min_estimators, max_estimators+1):
    clfB.set_params(n_estimators=i)
    clfB.fit(XB,yB)
    oob_error = 1-clfB.oob_score_
    error_rate.append((i, oob_error))
error_rate = np.array(error_rate)
plt.plot(error_rate[:,0], error_rate[:,1])
plt.xlim(min_estimators, max_estimators)
plt.title('OOB Error versus Number of Estimators for Clinical Only')
plt.xlabel('n_estimators')
plt.ylabel('OOB error rate')
plt.show()
nest = error_rate[:,0] 
err = error_rate[:,1]
n_estimatorsB = int(nest[np.where(err == np.min(err))[0][0]])

clfC = RandomForestClassifier(warm_start=True, max_features=None, 
                             oob_score=True, max_depth=max_depth, 
                             random_state=RANDOM_STATE)
error_rate = []
for i in range(min_estimators, max_estimators+1):
    clfC.set_params(n_estimators=i)
    clfC.fit(XC,yC)
    oob_error = 1-clfC.oob_score_
    error_rate.append((i, oob_error))
error_rate = np.array(error_rate)
plt.plot(error_rate[:,0], error_rate[:,1])
plt.xlim(min_estimators, max_estimators)
plt.title('OOB Error versus Number of Estimators for Clinical Only')
plt.xlabel('n_estimators')
plt.ylabel('OOB error rate')
plt.show()
nest = error_rate[:,0] 
err = error_rate[:,1]
n_estimatorsC = int(nest[np.where(err == np.min(err))[0][0]])


print n_estimatorsA, n_estimatorsB, n_estimatorsC

<h2>Measure corss-validation scores</h2>

In [ ]:
# Preprocessing
classifierA = RandomForestClassifier(n_estimators=n_estimatorsA, max_depth=max_depth, random_state=RANDOM_STATE,oob_score=True)
classifierB = RandomForestClassifier(n_estimators=n_estimatorsB, max_depth=max_depth, random_state=RANDOM_STATE, oob_score=True)
classifierC = RandomForestClassifier(n_estimators=n_estimatorsC, max_depth=max_depth, random_state=RANDOM_STATE, oob_score=True)

sss = StratifiedShuffleSplit(n_splits=NUM_BOOTSTRAPS, test_size=0.2, random_state=RANDOM_STATE)

print 'Generating feature matrices ...'
# Load data for CLINICAL ONLY
dfA = pd.read_csv('../data/qPET_feature_matrix_clinical_only.csv')
dfA_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_only.csv')
drop_idx = np.where(np.array(dfA.outcomeLatest) > 4)[0]
drop_test_idx = np.where(np.array(dfA_test.outcomeLatest) > 4)[0]
dfA = dfA.drop(dfA.index[drop_idx])
dfA_test = dfA_test.drop(dfA_test.index[drop_test_idx])

# Generate feature matrix for training and testing data
# Training
X1 = np.array(dfA[dfA.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X1_labels = np.array(dfA.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
# Testing
X1_test = np.array(dfA_test[dfA_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X1_test_labels = np.array(dfA_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))

# Load data for SSP ONLY
dfB = pd.read_csv('../data/qPET_feature_matrix_clinical_3dssp.csv')
dfB_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_3dssp.csv')
drop_idx = np.where(np.array(dfB.outcomeLatest) > 4)[0]
drop_test_idx = np.where(np.array(dfB_test.outcomeLatest) > 4)[0]
dfB = dfB.drop(dfB.index[drop_idx])
dfB_test = dfB_test.drop(dfB_test.index[drop_test_idx])

# Generate feature matrix for training and testing data
# Training
X2 = np.array(dfB[dfB.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X2_labels = np.array(dfB.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X2 = X2[:,SSP_features]
X2_labels = X2_labels[SSP_features]
# Testing
X2_test = np.array(dfB_test[dfB_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X2_test_labels = np.array(dfB_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X2_test = X2_test[:,SSP_features]
X2_test_labels = X2_test_labels[SSP_features]

# Load data for ASYMMETRY ONLY
dfC = pd.read_csv('../data/qPET_feature_matrix_clinical_voxel_ai.csv')
dfC_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_voxel_ai.csv')
drop_idx = np.where(np.array(dfC.outcomeLatest) > 4)[0]
drop_test_idx = np.where(np.array(dfC_test.outcomeLatest) > 4)[0]
dfC = dfC.drop(dfC.index[drop_idx])
dfC_test = dfC_test.drop(dfC_test.index[drop_test_idx])

# Generate feature matrix for training testing data
# Training
X3 = np.array(dfC[dfC.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X3_labels = np.array(dfC.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X3 = X3[:,ASYMM_features]
X3_labels = X3_labels[ASYMM_features]
# Testing
X3_test = np.array(dfC_test[dfC_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X3_test_labels = np.array(dfC_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X3_test = X3_test[:,ASYMM_features]
X3_test_labels = X3_test_labels[ASYMM_features]

# Generate outcome variable
outcome_threshold = 2 # i.e. Engel 1B
y = np.array(dfA.outcomeLatest)
y_test = np.array(dfA_test.outcomeLatest)
y[y<outcome_threshold] = 0
y[y>=outcome_threshold] = 1
y_test[y_test<outcome_threshold] = 0
y_test[y_test>=outcome_threshold] = 1

In [ ]:
columns=['Clinical Variable(s)','CV 5-Fold AUC_1','Out Of Bag (OOB1) Error',
         'Quantitative Variables','CV 5-Fold AUC_2','Out Of Bag (OOB2) Error',
         'AUC Difference 95% C.I.','OOB Difference 95% C.I.'
        ]
results = pd.DataFrame(
    {'Clinical Variable(s)':[],
     'CV 5-Fold AUC_1':[],
     'Out Of Bag (OOB1) Error':[],
     'Quantitative Variables':[],
     'CV 5-Fold AUC_2':[],
     'Out Of Bag (OOB2) Error':[],
     'AUC Difference 95% C.I.':[],
     'OOB Difference 95% C.I.':[]
             },
    columns=columns
)

OPTIONS = {
    1:[np.arange(20),'EEG+MRI+PET'],
    2:[np.arange(12),'EEG'],
    3:[np.arange(12,14),'MRI'],
    4:[np.arange(13,20),'PET'],
}

for OPTION,(clinical_variable_idx,variables_list) in OPTIONS.items():
    # Generate feature matrix for CLINICAL ONLY
    XA = X1[:,clinical_variable_idx]
    XA_labels = X1_labels[clinical_variable_idx]
    yA = y

    # Generate feature matrix for SSP alone
    XB = X2
    XB_labels = X2_labels
    XB_test = X2_test
    XB_test_labels = X2_test_labels
    yB = y

    # Generate bootstrap jobs
    jobs = []
    out = []
    for job_iter,(train, test) in enumerate(sss.split(XA,yA)):
        jobs.append((job_iter,train,test,XA,yA,XB,yB,classifierA,classifierB))
    # Run all jobs
    return_list = pool.map(_helper,jobs)
    # Compute OOB/AUC for bootstraps
    for res in return_list:
        probas_A,yA_test,probas_B,yB_test,classifierA_oob_score_,classifierB_oob_score_ = res
        fprA, tprA, thresholds = roc_curve(yA_test, probas_A[:, 1])
        fprB, tprB, thresholds = roc_curve(yB_test, probas_B[:, 1])
        roc_aucA = auc(fprA, tprA)
        roc_aucB = auc(fprB, tprB)
        out.append([roc_aucA,roc_aucB,classifierA_oob_score_,classifierB_oob_score_])
    out21 = np.array(out)

    # Generate feature matrix for SSP and CLINICAL
    XB = np.hstack((X1,X2))
    XB_labels = np.hstack((X1_labels,X2_labels))
    XB_test = np.hstack((X1_test,X2_test))
    XB_test_labels = np.hstack((X1_test_labels,X2_test_labels))

    # Generate bootstrap jobs
    jobs = []
    out = []
    for job_iter,(train, test) in enumerate(sss.split(XA,yA)):
        jobs.append((job_iter,train,test,XA,yA,XB,yB,classifierA,classifierB))
    # Run all jobs
    return_list = pool.map(_helper,jobs)
    # Compute OOB/AUC for bootstraps
    for res in return_list:
        probas_A,yA_test,probas_B,yB_test,classifierA_oob_score_,classifierB_oob_score_ = res
        fprA, tprA, thresholds = roc_curve(yA_test, probas_A[:, 1])
        fprB, tprB, thresholds = roc_curve(yB_test, probas_B[:, 1])
        roc_aucA = auc(fprA, tprA)
        roc_aucB = auc(fprB, tprB)
        out.append([roc_aucA,roc_aucB,classifierA_oob_score_,classifierB_oob_score_])
    out31 = np.array(out)

    # Compute statistics
    AUC1 = np.mean(out21[:,0])
    OOB1 = np.mean(out21[:,2])
    AUC2 = np.mean(out21[:,1])
    OOB2 = np.mean(out21[:,3])
    AUC3 = np.mean(out31[:,1])
    OOB3 = np.mean(out31[:,3])
    CI_AUC2_minus_AUC1 = tuple(map(lambda x: np.percentile(out21[:,1] - out21[:,0],x), [2.5, 97.5]))
    CI_AUC3_minus_AUC1 = tuple(map(lambda x: np.percentile(out31[:,1] - out31[:,0],x), [2.5, 97.5]))
    CI_OOB2_minus_OOB1 = tuple(map(lambda x: np.percentile(out21[:,3] - out21[:,2],x), [2.5, 97.5]))
    CI_OOB3_minus_OOB1 = tuple(map(lambda x: np.percentile(out31[:,3] - out31[:,2],x), [2.5, 97.5]))


    results = results.append(
        pd.DataFrame(
            [
                [variables_list,AUC1,OOB1,'%s+SSP'%(variables_list), AUC2, OOB2, CI_AUC2_minus_AUC1, CI_OOB2_minus_OOB1],
                [variables_list,AUC1,OOB1,'SSP only', AUC3, OOB3, CI_AUC3_minus_AUC1, CI_OOB3_minus_OOB1]
            ],
            columns=columns
        ),
        ignore_index=True
    )
cm = sns.light_palette("green",as_cmap=True)
results.style.background_gradient(cmap=cm,high=1.0)

In [ ]:
columns=['Clinical Variable(s)','CV 5-Fold AUC_1','Out Of Bag (OOB1) Error',
         'Quantitative Variables','CV 5-Fold AUC_2','Out Of Bag (OOB2) Error',
         'AUC Difference 95% C.I.','OOB Difference 95% C.I.'
        ]
results = pd.DataFrame(
    {'Clinical Variable(s)':[],
     'CV 5-Fold AUC_1':[],
     'Out Of Bag (OOB1) Error':[],
     'Quantitative Variables':[],
     'CV 5-Fold AUC_2':[],
     'Out Of Bag (OOB2) Error':[],
     'AUC Difference 95% C.I.':[],
     'OOB Difference 95% C.I.':[]
             },
    columns=columns
)

OPTIONS = {
    1:[np.arange(20),'EEG+MRI+PET'],
    2:[np.arange(12),'EEG'],
    3:[np.arange(12,14),'MRI'],
    4:[np.arange(13,20),'PET'],
}

for OPTION,(clinical_variable_idx,variables_list) in OPTIONS.items():
    # Generate feature matrix for CLINICAL ONLY
    XA = X1[:,clinical_variable_idx]
    XA_labels = X1_labels[clinical_variable_idx]
    yA = y

    # Generate feature matrix for SSP alone
    XC = X3
    XC_labels = X3_labels
    XC_test = X3_test
    XC_test_labels = X3_test_labels
    yC = y

    # Generate bootstrap jobs
    jobs = []
    out = []
    for job_iter,(train, test) in enumerate(sss.split(XA,yA)):
        jobs.append((job_iter,train,test,XA,yA,XC,yC,classifierA,classifierC))
    # Run all jobs
    return_list = pool.map(_helper,jobs)
    # Compute OOB/AUC for bootstraps
    for res in return_list:
        probas_A,yA_test,probas_C,yC_test,classifierA_oob_score_,classifierC_oob_score_ = res
        fprA, tprA, thresholds = roc_curve(yA_test, probas_A[:, 1])
        fprC, tprC, thresholds = roc_curve(yC_test, probas_C[:, 1])
        roc_aucA = auc(fprA, tprA)
        roc_aucC = auc(fprC, tprC)
        out.append([roc_aucA,roc_aucC,classifierA_oob_score_,classifierC_oob_score_])
    out21 = np.array(out)

    # Generate feature matrix for SSP and CLINICAL
    XC = np.hstack((X1,X3))
    XC_labels = np.hstack((X1_labels,X3_labels))
    XC_test = np.hstack((X1_test,X3_test))
    XC_test_labels = np.hstack((X1_test_labels,X3_test_labels))

    # Generate bootstrap jobs
    jobs = []
    out = []
    for job_iter,(train, test) in enumerate(sss.split(XA,yA)):
        jobs.append((job_iter,train,test,XA,yA,XC,yB,classifierA,classifierB))
    # Run all jobs
    return_list = pool.map(_helper,jobs)
    # Compute OOB/AUC for bootstraps
    for res in return_list:
        probas_A,yA_test,probas_C,yC_test,classifierA_oob_score_,classifierC_oob_score_ = res
        fprA, tprA, thresholds = roc_curve(yA_test, probas_A[:, 1])
        fprC, tprC, thresholds = roc_curve(yC_test, probas_C[:, 1])
        roc_aucA = auc(fprA, tprA)
        roc_aucC = auc(fprC, tprC)
        out.append([roc_aucA,roc_aucC,classifierA_oob_score_,classifierC_oob_score_])
    out31 = np.array(out)

    # Compute statistics
    AUC1 = np.mean(out21[:,0])
    OOB1 = np.mean(out21[:,2])
    AUC2 = np.mean(out21[:,1])
    OOB2 = np.mean(out21[:,3])
    AUC3 = np.mean(out31[:,1])
    OOB3 = np.mean(out31[:,3])
    CI_AUC2_minus_AUC1 = tuple(map(lambda x: np.percentile(out21[:,1] - out21[:,0],x), [2.5, 97.5]))
    CI_AUC3_minus_AUC1 = tuple(map(lambda x: np.percentile(out31[:,1] - out31[:,0],x), [2.5, 97.5]))
    CI_OOB2_minus_OOB1 = tuple(map(lambda x: np.percentile(out21[:,3] - out21[:,2],x), [2.5, 97.5]))
    CI_OOB3_minus_OOB1 = tuple(map(lambda x: np.percentile(out31[:,3] - out31[:,2],x), [2.5, 97.5]))


    results = results.append(
        pd.DataFrame(
            [
                [variables_list,AUC1,OOB1,'%s+Asymmetry'%(variables_list), AUC2, OOB2, CI_AUC2_minus_AUC1, CI_OOB2_minus_OOB1],
                [variables_list,AUC1,OOB1,'Asymmetry only', AUC3, OOB3, CI_AUC3_minus_AUC1, CI_OOB3_minus_OOB1]
            ],
            columns=columns
        ),
        ignore_index=True
    )
cm = sns.light_palette("green",as_cmap=True)
results.style.background_gradient(cmap=cm,high=1.0)

<h2>Feature improtances (Table 3)</h2>

In [ ]:
# Load data for CLINICAL ONLY
dfA = pd.read_csv('../data/qPET_feature_matrix_clinical_only.csv')
dfA_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_only.csv')
drop_idx = np.where(np.array(dfA.outcomeLatest) > 4)[0]
drop_test_idx = np.where(np.array(dfA_test.outcomeLatest) > 4)[0]
dfA = dfA.drop(dfA.index[drop_idx])
dfA_test = dfA_test.drop(dfA_test.index[drop_test_idx])

# Generate feature matrix for training and testing data
# Training
X1 = np.array(dfA[dfA.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X1_labels = np.array(dfA.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
# Testing
X1_test = np.array(dfA_test[dfA_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X1_test_labels = np.array(dfA_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))

# Load data for SSP ONLY
dfB = pd.read_csv('../data/qPET_feature_matrix_clinical_3dssp.csv')
dfB_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_3dssp.csv')
drop_idx = np.where(np.array(dfB.outcomeLatest) > 4)[0]
drop_test_idx = np.where(np.array(dfB_test.outcomeLatest) > 4)[0]
dfB = dfB.drop(dfB.index[drop_idx])
dfB_test = dfB_test.drop(dfB_test.index[drop_test_idx])

# Generate feature matrix for training and testing data
# Training
X2 = np.array(dfB[dfB.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X2_labels = np.array(dfB.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X2 = X2[:,SSP_features]
X2_labels = X2_labels[SSP_features]
# Testing
X2_test = np.array(dfB_test[dfB_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X2_test_labels = np.array(dfB_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X2_test = X2_test[:,SSP_features]
X2_test_labels = X2_test_labels[SSP_features]

# Load data for ASYMMETRY ONLY
dfC = pd.read_csv('../data/qPET_feature_matrix_clinical_voxel_ai.csv')
dfC_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_voxel_ai.csv')
drop_idx = np.where(np.array(dfC.outcomeLatest) > 4)[0]
drop_test_idx = np.where(np.array(dfC_test.outcomeLatest) > 4)[0]
dfC = dfC.drop(dfC.index[drop_idx])
dfC_test = dfC_test.drop(dfC_test.index[drop_test_idx])

# Generate feature matrix for training testing data
# Training
X3 = np.array(dfC[dfC.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X3_labels = np.array(dfC.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X3 = X3[:,ASYMM_features]
X3_labels = X3_labels[ASYMM_features]
# Testing
X3_test = np.array(dfC_test[dfC_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X3_test_labels = np.array(dfC_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X3_test = X3_test[:,ASYMM_features]
X3_test_labels = X3_test_labels[ASYMM_features]

# Generate outcome variable
outcome_threshold = 2 # i.e. Engel 1B
y = np.array(dfA.outcomeLatest)
y_test = np.array(dfA_test.outcomeLatest)
# y[y<outcome_threshold] = 0
# y[y>=outcome_threshold] = 1
# y_test[y_test<outcome_threshold] = 0
# y_test[y_test>=outcome_threshold] = 1

# Run predictions
# Generate feature matrix for CLINICAL ONLY
XA = X1
XA_labels = X1_labels
XA_test = X1_test
XA_test_labels = X1_test_labels
yA = y

# Generate feature matrix for SSP and CLINICAL
XB = np.hstack((X1,X2))
XB_labels = np.hstack((X1_labels,X2_labels))
XB_test = np.hstack((X1_test,X2_test))
XB_test_labels = np.hstack((X1_test_labels,X2_test_labels))
yB = y

# Generate feature matrix for SSP and CLINICAL
XC = np.hstack((X1,X3))
XC_labels = np.hstack((X1_labels,X3_labels))
XC_test = np.hstack((X1_test,X3_test))
XC_test_labels = np.hstack((X1_test_labels,X3_test_labels))
yC = y


In [ ]:
robjects.globalenv['RANDOM_STATE'] = RANDOM_STATE

# Generate final form of Table 2
columns=[
    'Feature',
    'Feature Importance',
    'Univariate Odds Ratio per Unit\n Increase in Seizure Recurrence [95% CI, p]'
        ]
results = pd.DataFrame(
    {'Feature':[],
     'Feature Importance':[],
     'Univariate Odds Ratio per Unit\n Increase in Seizure Recurrence [95% CI, p]':[]
             },
    columns=columns
)

for X,y, X_labels in [(XA,yA,XA_labels), (XB,yB,XB_labels), (XC,yC,XC_labels)]:
    # Run feature importance (IncMSE) for random forests
    robjects.globalenv['X'] = X
    robjects.globalenv['y'] = y
    summary = robjects.r['summary']

    robjects.r('''
        set.seed(RANDOM_STATE)
        library(randomForest)        
        rf <- randomForest(y ~ ., data=X, importance=TRUE)
        X_importances <- importance(rf)    
    ''')
    X_importances = robjects.r['X_importances']
    feature_order = np.argsort(-X_importances[:,0])

    # Run univariate odds ratio computation
    for feature in feature_order:
        robjects.globalenv['feature'] = X[:,feature]    
        robjects.r('''        
            set.seed(RANDOM_STATE)
            require(MASS)        
            myfit <- polr(as.factor(y) ~ feature, Hess=TRUE)
            (ctable <- coef(summary(myfit)))            
            p <- pnorm(abs(ctable[,"t value"]), lower.tail=FALSE)*2
            (ctable <- cbind(ctable, "p value" = p))
            (ci <- confint(myfit))
            myfit_ci <-exp(cbind(OR = coef(myfit), ci))            
        ''')
        myfit = robjects.r['myfit']
        myfit_ci = robjects.r['myfit_ci']
        pval = robjects.r['p'][0]
        label = X_labels[feature]
        try:
            label = label.replace(label.split('_')[-1],ssp_labels[int(label.split('_')[-1])])
        except Exception:
            pass

        if X_importances[feature,0] <= 0.0:
            continue
        
        if pval < 0.001:
            pval = '%0.3f ***'%pval
        elif pval < 0.01:
            pval = '%0.3f **'%pval
        elif pval < 0.05:
            pval = '%0.3f *'%pval
        elif pval < 0.1:
            pval = '%0.3f .'%pval
        else:
            pval = '%0.2f'%pval
        
        results = results.append(
            pd.DataFrame(
                [
                    [
                        label,
                        '%0.2f'%X_importances[feature,0], 
                        '%0.2f [%0.2f - %0.2f, %s]'%(
                            myfit_ci[0,0],
                            myfit_ci[0,1], myfit_ci[1,1], 
                            pval
                        )
                    ]
                ],
                columns=columns
            ),
            ignore_index=True
        )
    results = results.append(
        pd.DataFrame(
            [
                [
                    '',
                    '', 
                    ''
                ]
            ],
            columns=columns
        ),
        ignore_index=True
    )
# cm = sns.light_palette("green",as_cmap=True)
# results.style.background_gradient(cmap=cm,high=1.0,low=0.0)
display(results)
results.to_csv('results_Section5.csv')

<h2>Validation on testing set (Table 4)</h2>

In [ ]:
# Create feature matrix for training and testing
# Load data for CLINICAL ONLY
dfA = pd.read_csv('../data/qPET_feature_matrix_clinical_only.csv')
dfA_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_only.csv')
drop_idx = np.where(np.array(dfA.outcomeLatest) > 4)[0]
drop_test_idx = np.where(np.array(dfA_test.outcomeLatest) > 4)[0]
dfA = dfA.drop(dfA.index[drop_idx])
dfA_test = dfA_test.drop(dfA_test.index[drop_test_idx])

# Generate feature matrix for training and testing data
# Training
X1 = np.array(dfA[dfA.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X1_labels = np.array(dfA.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
# Testing
X1_test = np.array(dfA_test[dfA_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X1_test_labels = np.array(dfA_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))

# Load data for SSP ONLY
dfB = pd.read_csv('../data/qPET_feature_matrix_clinical_3dssp.csv')
dfB_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_3dssp.csv')
drop_idx = np.where(np.array(dfB.outcomeLatest) > 4)[0]
drop_test_idx = np.where(np.array(dfB_test.outcomeLatest) > 4)[0]
dfB = dfB.drop(dfB.index[drop_idx])
dfB_test = dfB_test.drop(dfB_test.index[drop_test_idx])

# Generate feature matrix for training and testing data
# Training
X2 = np.array(dfB[dfB.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X2_labels = np.array(dfB.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X2 = X2[:,SSP_features]
X2_labels = X2_labels[SSP_features]
# Testing
X2_test = np.array(dfB_test[dfB_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X2_test_labels = np.array(dfB_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X2_test = X2_test[:,SSP_features]
X2_test_labels = X2_test_labels[SSP_features]

# Load data for ASYMMETRY ONLY
dfC = pd.read_csv('../data/qPET_feature_matrix_clinical_voxel_ai.csv')
dfC_test = pd.read_csv('../data/qPET_Validation_feature_matrix_clinical_voxel_ai.csv')
drop_idx = np.where(np.array(dfC.outcomeLatest) > 4)[0]
drop_test_idx = np.where(np.array(dfC_test.outcomeLatest) > 4)[0]
dfC = dfC.drop(dfC.index[drop_idx])
dfC_test = dfC_test.drop(dfC_test.index[drop_test_idx])

# Generate feature matrix for training testing data
# Training
X3 = np.array(dfC[dfC.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X3_labels = np.array(dfC.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X3 = X3[:,ASYMM_features]
X3_labels = X3_labels[ASYMM_features]
# Testing
X3_test = np.array(dfC_test[dfC_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest'])])
X3_test_labels = np.array(dfC_test.columns.difference(['Unnamed: 0','id','outcome12','outcome24','outcome6','outcomeLatest']))
X3_test = X3_test[:,ASYMM_features]
X3_test_labels = X3_test_labels[ASYMM_features]

# Generate outcome variable
outcome_threshold = 2 # i.e. Engel 1B
y = np.array(dfA.outcomeLatest)
y_test = np.array(dfA_test.outcomeLatest)
y[y<outcome_threshold] = 0
y[y>=outcome_threshold] = 1
y_test[y_test<outcome_threshold] = 0
y_test[y_test>=outcome_threshold] = 1

# Run predictions
# Generate feature matrix for CLINICAL ONLY
XA = X1
XA_labels = X1_labels
XA_test = X1_test
XA_test_labels = X1_test_labels

# Generate feature matrix for SSP and CLINICAL
XB = np.hstack((X1,X2))
XB_labels = np.hstack((X1_labels,X2_labels))
XB_test = np.hstack((X1_test,X2_test))
XB_test_labels = np.hstack((X1_test_labels,X2_test_labels))

# Generate feature matrix for SSP and CLINICAL
XC = np.hstack((X1,X3))
XC_labels = np.hstack((X1_labels,X3_labels))
XC_test = np.hstack((X1_test,X3_test))
XC_test_labels = np.hstack((X1_test_labels,X3_test_labels))

# Load library in R
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
pROC = importr('pROC')

# Train classifier and apply to validation test set
classifierA = RandomForestClassifier(
    n_estimators=n_estimatorsA, 
    max_depth=max_depth, 
    random_state=RANDOM_STATE,
    oob_score=True)
classifierB = RandomForestClassifier(
    n_estimators=n_estimatorsB, 
    max_depth=max_depth, 
    random_state=RANDOM_STATE,
    oob_score=True)
classifierC = RandomForestClassifier(
    n_estimators=n_estimatorsC, 
    max_depth=max_depth, 
    random_state=RANDOM_STATE,
    oob_score=True)

probas = classifierA.fit(XA, y).predict_proba(XA_test)
predsA = robjects.FloatVector(probas[:,1])
labelsA = robjects.IntVector(y_test)
probas = classifierB.fit(XB, y).predict_proba(XB_test)
predsB = robjects.FloatVector(probas[:,1])
labelsB = robjects.IntVector(y_test)
probas = classifierC.fit(XC, y).predict_proba(XC_test)
predsC = robjects.FloatVector(probas[:,1])
labelsC = robjects.IntVector(y_test)


# Copy from python workspace to R workspace
robjects.globalenv["predsA"] = predsA
robjects.globalenv["labelsA"] = labelsA
robjects.globalenv["predsB"] = predsB
robjects.globalenv["labelsB"] = labelsB
robjects.globalenv["predsC"] = predsC
robjects.globalenv["labelsC"] = labelsC

# Run pROC.roc and pROC.roc.test in R
robjects.r('''
    predsA<-as.numeric(unlist(predsA))
    labelsA<-as.numeric(unlist(labelsA))
    predsB<-as.numeric(unlist(predsB))
    labelsB<-as.numeric(unlist(labelsB))
    predsC<-as.numeric(unlist(predsC))
    labelsC<-as.numeric(unlist(labelsC))

    library(pROC)
    
    roc1 <- roc(labelsA, predsA, percent=FALSE, 
    smooth=TRUE, ci=TRUE, boot.n=100, ci.alpha=0.95, 
    stratified=TRUE,plot=FALSE,grid=FALSE,print.auc=FALSE, show.thres=TRUE, col='red')
    roc2 <- roc(labelsB, predsB, percent=FALSE, 
    smooth=TRUE, ci=TRUE, boot.n=100, ci.alpha=0.95, 
    stratified=TRUE,plot=FALSE,grid=FALSE,print.auc=FALSE, show.thres=TRUE, col='red')
    roc3 <- roc(labelsC, predsC, percent=FALSE, 
    smooth=TRUE, ci=TRUE, boot.n=100, ci.alpha=0.95, 
    stratified=TRUE,plot=FALSE,grid=FALSE,print.auc=FALSE, show.thres=TRUE, col='red')
    roc_test12<-roc.test(roc1,roc2)
    roc_test13<-roc.test(roc1,roc3)
    test_accuracy1<-coords(roc1, "best", ret=c("specificity","sensitivity","accuracy"), best.method=c("youden","closest.topleft"));
    test_accuracy2<-coords(roc2, "best", ret=c("specificity","sensitivity","accuracy"), best.method=c("youden","closest.topleft"));
    test_accuracy3<-coords(roc3, "best", ret=c("specificity","sensitivity","accuracy"), best.method=c("youden","closest.topleft"));
''')

# Generate final form of Table 4 
columns=['Model','AUC','Accuracy','Sensitivity','Specificity','p-value'
        ]
results = pd.DataFrame(
    {'Model':[],
     'AUC':[],
     'Accuracy':[],
     'Sensitivity':[],
     'Specificity':[],
     'p-value':[]
             },
    columns=columns
)

results = results.append(
    pd.DataFrame(
        [
            [
                'Model 1',
                robjects.r["roc1"].rx2('auc')[0],
                robjects.r["test_accuracy1"][2],
                robjects.r["test_accuracy1"][1],
                robjects.r["test_accuracy1"][0],
                np.nan
            ],
            [
                'Model 2',
                robjects.r["roc2"].rx2('auc')[0],
                robjects.r["test_accuracy2"][2],
                robjects.r["test_accuracy2"][1],
                robjects.r["test_accuracy2"][0],
                robjects.r["roc_test12"].rx2('p.value')[0]
            ],
            [
                'Model 3',
                robjects.r["roc3"].rx2('auc')[0],
                robjects.r["test_accuracy3"][2],
                robjects.r["test_accuracy3"][1],
                robjects.r["test_accuracy3"][0],
                robjects.r["roc_test13"].rx2('p.value')[0]
            ]
        ],
        columns=columns
    ),
    ignore_index=True
)
cm = sns.light_palette("green",as_cmap=True)
results.style.background_gradient(cmap=cm,high=1.0,low=0.0)